In [ ]:
#'''
# **************************************************************************************************************** #
#*****************************************  IDB - AUG Data Analytics  ******************************************** #
# **************************************************************************************************************** #
#
#-- Notebook Number: 03.2 - NLP Processing English (Loans and TCs - Stanza) (workpaper)
#-- Title: Digital Transformation Advisory
#-- Audit Segment: 
#-- Continuous Auditing: Yes
#-- System(s): pdf files
#-- Description:  
#                - Process the annotated text from loans and TCs, generating terms and tokens
#                - Results are filtered and lemmatized using Stanza NLP
#                
#                
#
#-- @authors:  Emiliano Colina <emilianoco@iadb.org>
#-- Version:  1.2
#-- Last Update: 01/15/2021
#-- Last Revision Date: 01/14/2021 - Emiliano Colina <emilianoco@iadb.org> 
#                                    

# **************************************************************************************************************** #
#'''

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
import os
import re, numpy as np, pandas as pd
from pprint import pprint
import joblib

In [ ]:
from collections import Counter

#### Environment Setup

In [ ]:
# Set working directory
main_dir = "C:\\Users\\emilianoco\\Desktop\\2020"
data_dir = "/Digital_Transformation"


os.chdir(main_dir + data_dir) # working directory set
print('Working folder set to: ' + os.getcwd()) # working directory check

In [ ]:
# download the es_core_news_lg:
#!python -m spacy download es_core_news_lg

In [ ]:
# -------- START v1.2 ----------- #

In [ ]:
%%time
# v1.2: load the dataframe from the previous notebook containing the annotated text
df_base = joblib.load('./output/nlp_2021-01-15_spacy_annotated_english.joblib.bz2')
df_base.head()

In [ ]:
df_base['textacy_processing'][1]

### 1. Textacy (v1.2)

In [ ]:
import textacy

# Load Spacy English model in Textacy:
en = textacy.load_spacy_lang('en_core_web_lg')

### 2. List of Terms: n-Grams extraction (v1.2)

#### 2.1. List of Terms Generation

In [ ]:
df_base['alt2_list_terms_base'] = ''
df_base['alt2_list_terms'] = ''
df_base.head()

In [ ]:
%%time
for index, row in df_base.iterrows():
    #print('Processing index:', str(index))
    df_base.at[index, 'textacy_processing'] = textacy.make_spacy_doc(df_base.extracted[index], lang=en)
    
df_base.head()

In [ ]:
%%time
for index, row in df_base.iterrows():
    print('processing:', index)
    
    #generate terms (returns a generator):
    terms_list = df_base['textacy_processing'][index]._.to_terms_list(ngrams=(2, 3, 4, 5, 6), entities=False, normalize="lower", weighting="count", as_strings=True, filter_stops=True, filter_punct=True, filter_nums=True, include_pos=['PROPN', 'NOUN', 'ADJ', 'ADP'], min_freq=2)

    #convert to list:
    terms_list = list(terms_list)

    #create dictio {term, count}:
    resultado_pre = Counter([item.lower() for item in terms_list])
    
    #store result:
    df_base.at[index, 'alt2_list_terms_base'] = resultado_pre
    print('done!')

#### 2.2. Terms Lemmatization using Stanza

In [ ]:
import stanza

stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma', lang='en', use_gpu=True) 

In [ ]:
df_base.head()

In [ ]:
%%time
for index, row in df_base.iterrows():
    resultado = []
    
    print('processing:', index)
    #compute desired pos and join terms:
    for k,v in df_base['alt2_list_terms_base'][index].items():
        doc = stNLP(k); term = ('_'.join([word.lemma for sent in doc.sentences for word in sent.words if word.pos in ['NOUN', 'ADJ', 'PROPN', 'PUNCT']]), v)
        if '_' in term[0]:
            resultado.append(term)
    
    #merge repetitive terms and counts :
    resultado_pre = {x[0] for x in resultado}
    resultado_post = [(i,sum(x[1] for x in resultado if x[0] == i)) for i in resultado_pre]
    
    #store:
    df_base.at[index, 'alt2_list_terms'] = resultado_post
    
    del resultado; del resultado_pre; del resultado_post
    print('done!')
    

In [ ]:
df_base.head()

In [ ]:
df_base.shape

In [ ]:
# x x x x x x x x 

### 3. NLP Token extraction and processing (v1.2)

In [ ]:
import gensim
# Tokenize Sentences and Clean
def sent_to_words(sentences):
    for sent in sentences:
        #sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        #sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=False) #modificado
        yield(sent) 

In [ ]:
# Main functions
def remove_stopwords(texts, stop_words):
    return [[word for word in gensim.utils.simple_preprocess(str(doc), deacc=False) if word not in stop_words] for doc in texts]

#
def lemmatization(texts, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP']):
    texts_out = []
    for sent in texts:
        doc = stNLP(" ".join(sent)) 
        texts_out.append([word.lemma.lower() for sent in doc.sentences for word in sent.words if word.pos in allowed_postags])
    # remove stopwords once more after lemmatization
    #texts_out = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]   
    return texts_out

In [ ]:
# v1.2: Get the "extracted" content and convert to list
data = df_base['extracted'].values.tolist()
data_words = list(sent_to_words(data))
pprint(data_words[:1])

In [ ]:
%%time
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words, stop_words)

# Data Lemmatized
data_lemmatized = lemmatization(data_words, allowed_postags=['PROPN', 'NOUN', 'ADJ'])

In [ ]:
## store all tokens:
df_base['alt2_tokens_pre_clean'] = data_lemmatized 

In [ ]:
# v1.2: Storing of intermediate results df_base, English, containing terms and tokens pre-cleaned process
f_df_base = 'nlp_df_base_tokens_terms_2021-01-15_english_pre-clean_v1.2.joblib'
joblib.dump(df_base, './output/' + f_df_base + '.bz2', compress=('bz2', 3))#

In [ ]:
import pickle

In [ ]:
## v1.2: Tokens (English)
with open('./output/nlp_operations_tokens_2021-01-15_english_pre-clean_v1.2.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized, filehandle)

In [ ]:
# -------- END v1.2 ----------- #

In [ ]:
df_base[['doc_type', 'language', 'OPERATION_NUMBER', 'DOCUMENT_REFERENCE']].to_excel('./output/operaciones_en_ingles.xlsx')

# **************************************************************************************************************** #
<br>
<br>
<br>

### 4. Clean-up

#### 4.1: Terms

In [ ]:
df_base.head()

In [ ]:
df_base['alt2_list_terms_expanded'] = ''

In [ ]:
%%time
#expand the list of tuples:
for index, row in df_base.iterrows():
    print('processing index:', index)
    df_base.at[index, 'alt2_list_terms_expanded'] = [k for (k,v) in df_base.alt2_list_terms[index] for count in range(v)]
    print('done!')

In [ ]:
terms_result = df_base.alt2_list_terms_expanded.to_list()

terms_grams = []
for i in range(len(terms_result)):
    for token in terms_result[i]:
        terms_grams.append(token)
        
print((len((terms_grams)),len(set(terms_grams))))

In [ ]:
terms_grams = Counter(terms_grams)
sort_orders_terms = sorted(terms_grams.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_terms:
    print(i[0], i[1])

#### 4.1.1: Terms to remove

In [ ]:
terms_to_remove = []
for i in range(0,len(sort_orders_terms)):
    if (sort_orders_terms[i][0].endswith('_iv') or sort_orders_terms[i][0].endswith('_ii') or sort_orders_terms[i][0].endswith('_us$') or sort_orders_terms[i][0].endswith('/') \
        or sort_orders_terms[i][0].endswith('.') or sort_orders_terms[i][0].endswith('_i') or sort_orders_terms[i][0].endswith('_iii') or sort_orders_terms[i][0].endswith('_”')\
        or sort_orders_terms[i][0].endswith('_a') or sort_orders_terms[i][0].startswith('“_') or sort_orders_terms[i][0].startswith('f._') or sort_orders_terms[i][0].startswith('a._') \
        or sort_orders_terms[i][0].startswith('b._') or sort_orders_terms[i][0].startswith('c._') or sort_orders_terms[i][0].startswith('d._') \
        or sort_orders_terms[i][0].startswith('e._') or sort_orders_terms[i][0].startswith('v._') or sort_orders_terms[i][0].startswith('i._') \
        or sort_orders_terms[i][0].startswith('g._') or sort_orders_terms[i][0].startswith('iv._') or sort_orders_terms[i][0].startswith('&_') \
        or sort_orders_terms[i][0].startswith('actividad/_') or sort_orders_terms[i][0].startswith('ct_') or sort_orders_terms[i][0].startswith('atn_/') \
        or sort_orders_terms[i][0].startswith('/_') or sort_orders_terms[i][0].startswith('ii.') or sort_orders_terms[i][0].startswith('iii_') or sort_orders_terms[i][0].startswith('iv_')\
        or sort_orders_terms[i][0].startswith('a_') or sort_orders_terms[i][0].endswith('_rev') or sort_orders_terms[i][0].startswith('x_') or sort_orders_terms[i][0].startswith('p_') \
        or sort_orders_terms[i][0].startswith('d_') or sort_orders_terms[i][0].startswith('enel_') or sort_orders_terms[i][0].endswith('_enel') or sort_orders_terms[i][0].endswith('_p') or sort_orders_terms[i][0].endswith('_d')\
        or sort_orders_terms[i][0].endswith('_figura') or sort_orders_terms[i][0].endswith('_sp') or sort_orders_terms[i][0].endswith('_cis') or sort_orders_terms[i][0].endswith('_csc') or sort_orders_terms[i][0].endswith('_cobit')\
        or sort_orders_terms[i][0].startswith('dela_') or sort_orders_terms[i][0].endswith('_dela') or sort_orders_terms[i][0].endswith('_nist') or sort_orders_terms[i][0].endswith('_cert') \
        or sort_orders_terms[i][0].endswith('_t') or sort_orders_terms[i][0].endswith('_m') or sort_orders_terms[i][0].startswith('m_') or sort_orders_terms[i][0].startswith('is_') or sort_orders_terms[i][0].startswith('for_')\
        or sort_orders_terms[i][0].startswith('and_') or sort_orders_terms[i][0].startswith('of_') or sort_orders_terms[i][0].startswith('or_') or sort_orders_terms[i][0].startswith('this_') or sort_orders_terms[i][0].startswith('does_')\
        or sort_orders_terms[i][0].startswith('are_') or sort_orders_terms[i][0].startswith('j_') or sort_orders_terms[i][0].startswith('c_') or sort_orders_terms[i][0].endswith('_is') \
        or sort_orders_terms[i][0].endswith('_be') or sort_orders_terms[i][0].endswith('_and') or sort_orders_terms[i][0].endswith('_are') or sort_orders_terms[i][0].endswith('_of')\
        or sort_orders_terms[i][0].endswith('_nº') or sort_orders_terms[i][0].endswith('_t(')
       ):
       
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove.append(sort_orders_terms[i][0])

In [ ]:
len(set(terms_to_remove))

In [ ]:
def num_there(s):
    return any(i.isdigit() for i in s)

# remove grmas containing digits:
for i in range(0,len(sort_orders_terms)):
    if (num_there(sort_orders_terms[i][0]) and not ('covid' in sort_orders_terms[i][0] or '2700' in sort_orders_terms[i][0] \
                                                    or 'p2p' in sort_orders_terms[i][0] or '5g' in sort_orders_terms[i][0] or 'co2' in sort_orders_terms[i][0])):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if ('follow' in sort_orders_terms[i][0]) or ('receive' in sort_orders_terms[i][0]) or ('et_al' in sort_orders_terms[i][0]) or \
        (sort_orders_terms[i][0].startswith('option'))  \
        or ('measure' in sort_orders_terms[i][0]) or ('ofmeasure' in sort_orders_terms[i][0]) or ('conclusion' in sort_orders_terms[i][0]) or ('_adaptational' in sort_orders_terms[i][0])\
        or ('sigla' in sort_orders_terms[i][0]) or ('abrev' in sort_orders_terms[i][0]): # or ('major' in sort_orders_terms[i][0]) or ('minor' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if ('paragraph' in sort_orders_terms[i][0] and not ('lista_revocación' in sort_orders_terms[i][0])):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        #terms_to_remove.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if (('▪' in sort_orders_terms[i][0]) or ('.' in sort_orders_terms[i][0]) or ('+' in sort_orders_terms[i][0]) or ('¶' in sort_orders_terms[i][0]) \
        or ('$' in sort_orders_terms[i][0]) or ('' in sort_orders_terms[i][0]) or ('=' in sort_orders_terms[i][0]) or ('/' in sort_orders_terms[i][0]) \
        or ('€' in sort_orders_terms[i][0])) and not ('.uy' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove.append(sort_orders_terms[i][0])

In [ ]:
for i in sort_orders_terms:
    if 'component' in i[0] and not 'critical' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'figur' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'relate' in i[0] or 'proyecto_ley' in i[0] or 'fundamental' in i[0] or 'incluido' in i[0] or 'asociado_' in i[0] or 'aproximación_' in i[0] \
        or 'caracterización' in i[0] or 'intencional_causado' in i[0] or 'referencia_apartado' in i[0] or 'unido_compromiso' in i[0] or 'extremo_extremo' in i[0] \
        or 'arte_gestión' in i[0] or 'protocolo_campo' in i[0] or 'estudio_trend' in i[0] or 'autoridad_relevante' in i[0] or 'latino_marco' in i[0] \
        or 'interamericano_integral' in i[0] or 'formado_situ' in i[0] or 'gobierno_seguridad' in i[0] or 'estudio_trend' in i[0] or 'cobertura_continu' in i[0] \
        or 'continuo_medio' in i[0] or 'incidente_impacto' in i[0] or 'sistema_categoría_alto' in i[0] or 'presente_guía' in i[0] or 'cibernético_divulgación' in i[0] \
        or 'relevante_' in i[0] or 'año_edad' in i[0] or 'financiado_' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if ('_x_' in i[0] or i[0].endswith('_x') or i[0].startswith('x_')) or ('_d_' in i[0] or i[0].endswith('_d') or i[0].startswith('d_')) or \
        ('_m_' in i[0] or i[0].endswith('_m') or i[0].startswith('m_')) or (i[0].endswith('_c') or i[0].startswith('c_') or '_c_' in i[0]):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].endswith('_l') or i[0].startswith('l_') or i[0].endswith('_del') or i[0].startswith('del_') or i[0].endswith('detailed') or i[0].startswith('detailed_') \
        or i[0].endswith('_n') or i[0].startswith('n_'):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].endswith('_ic') or i[0].startswith('ic_') or i[0].endswith('_r') or i[0].startswith('r_') or i[0].startswith('f_') or i[0].endswith('_v') or i[0].startswith('sa_') \
        or i[0].startswith('an_') or i[0].endswith('_an') or i[0].startswith('by_') or i[0].endswith('_by') or i[0].startswith('or_') or i[0].endswith('_or') \
        or i[0].startswith('can_') or i[0].endswith('_can') or i[0].startswith('to_') or i[0].endswith('_to') or i[0].startswith('cabo_') or i[0].endswith('_cabo') \
        or i[0].startswith('the_') or i[0].endswith('_the') or i[0].startswith('of_') or i[0].endswith('_of') or i[0].startswith('and_') or i[0].endswith('_and') \
        or '_and_' in i[0] or i[0].startswith('h_') or i[0].endswith('_b') or i[0].endswith('_nis') or i[0].endswith('_n') or i[0].startswith('one_') or i[0].endswith('_one') \
        or i[0].endswith('_af') or i[0].endswith('_aes') or i[0].endswith('_cc') or i[0].endswith('_sncti') or i[0].endswith('_cb') or i[0].endswith('_foppa'):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].endswith('_inc') or i[0].startswith('inc_') or ('_inc_' in i[0]):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].endswith('_int') or i[0].startswith('int_') or ('_int_' in i[0]):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].endswith('_') or i[0].startswith('_'):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'used' in i[0] or 'amplio_cobertura' in i[0] or 'definido' in i[0] or 'frecuencia_reportar' in i[0] or 'red_saro' in i[0] or 'brasil' in i[0] \
        or 'uruguay' in i[0] or 'trinidad' in i[0] or 'concerniente' in i[0] or 'corea_sur' in i[0] or 'ejemplo' in i[0] or 'electric_corp' in i[0] \
        or 'american_' in i[0] or 'lan_recuperac' in i[0] or 'forma_claro' in i[0] or 'costa_rico' in i[0] or 'proyecto_ley' in i[0] or 'ley_específico' in i[0] \
        or 'cuarto_nivel' in i[0] or 'paraguay' in i[0] or 'barbado' in i[0] or 'taller' in i[0] or 'nacional_contratación_público' in i[0] or 'nacional_inteligencia' in i[0] \
        or 'nacional_telecomunicación' in i[0] or 'aplicación_ens' in i[0] or 'instrumento_línea' in i[0] or 'evento_capacitación' in i[0] or 'curso_capacitación' in i[0] \
        or 'calidad_software_control' in i[0] or 'ciberseguridadriesgo' in i[0] or 'guyana' in i[0] or 'chile' in i[0] or 'argentin' in i[0] or 'peru' in i[0] \
        or 'ecuador' in i[0] or 'bolivia' in i[0] or 'guatemala' in i[0] or 'dominican_republic' in i[0] or 'país_caribe_oriental' in i[0]:    
        print(i[0], i[1])
        #terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if ('oportion' in i[0] and not 'oportunidad_innovación' in i[0]) or ('_financed_' in i[0]):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    #if i[0].endswith('_sector') or i[0].endswith('_level'):
    if i[0].startswith('sector') or i[0].startswith('level_'):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'userin' in i[0] or 'depropiedad' in i[0] or 'sensibilización_sensibilización' in i[0] or 'decomunicación' in i[0] or 'frecuencia_medición' in i[0] \
        or 'frecuencia_organización' in i[0] or 'respuesta_respuesta' in i[0] or 'mentalidad' in i[0] or '_habilidad' in i[0] or 'provisión_apropiación' in i[0] \
        or 'ubicado' in i[0] or 'guía_aproximación' in i[0] or 'socio_externo' in i[0] or 'conformidad_política' in i[0] \
        or 'hoja_ruta' in i[0] or 'red_csirto' in i[0]:
        print(i[0], i[1])
        #terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'j_k' in i[0] or 'f_g_' in i[0] or 'kg_hect' in i[0] or 'mg_ha' in i[0] or 't_ico' in i[0] or 'icac_ión' in i[0] or 'serv_ic_ios' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if '>' in i[0] or '<' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'which' in i[0] or 'where' in i[0] or 'when' in i[0] or 'what' in i[0] or 'that' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
curated_list = ['américa_latino', 'parte_interesado', 'nacional_ciberseguridad', 'anterior', 'sector_privado', 'sector_público', 'sector_eléctrico_américa'
'eléctrico_américa', 'ciberseguridad_sector_eléctrico_américa', 'preparación_ciberseguridad_sector_eléctrico', 'américa_latino', 'transmisiónuno', 'cooperación_formal', 'intelectual_legislación', 'eldesarrollo', 'usuario_confianza', 'procesalcontro', 'red_to'
'evaluación_cualitativo', 'sector_público', 'operador_subsector', 'actividad_gestión', 'actividad_operador', 'organización_', 'personal_línea', 'sistema_distribución',
'protecciónal', 'banco_mundial', 'porcentaje_penetración', 'sustantivacontra', 'línea_fuerza', 'deseguridad', 'tic_legislación', 'comprensión_usuario', 'sobreprotección', 'contenido', 'cibernéticad', 'identificación_organización', 'internet_persona', 'estrategia_organización', 'adquisición_estándar'
'privado_usuario', 'fuerza_orden', 'organización_contenido', 'eléctrico_américa', 'américa', 'usuario_protección', 'estación_eng', 'generación_distribución', 'imagen_operativo'
'sistema_generación', 'necesidad_negocio', 'riesgo_cadena', 'operativo_común', 'energía_eléctrico', 'imagen_operativo', 'gestión_fuerza', 'transmisión_operador', 'nivel_implementación', 'enjuiciamiento', 'capabilitie_type', 'subcategoría_nivel', 'remoto_red', 
'transmisión_operador_sistema_generación', 'referencia_requisito', 'subcategoría_prioridad', 'perfil_referencia', 'transmisiónuna', 'fuerza_laboral', 'recomendación_formulador'               
'entendimiento', 'nacional_seguridad_cibernético', 'red_to', 'evaluación', 'carrera_grado', 'autoridad_nacional_innovación', 'gubernamental_país', 'control_criptográfico_seguro_cibernético_estándar',
'oportunidad_estudio', 'nivel_preparación', 'tecnología_seguridad_cibernético_control_técnico', 'etapa_madurez', 'software_control_criptográfico_seguro_cibernético', 'resultado_operación_préstamo',
'seguridad_soberano', 'aspecto_ciberseguridad', 'control_técnico_seguridad_divulgación_responsable', 'desarrollo_estado_verde', 'importancia_ciberseguridad', 'fortalecimiento_capacidad', 'legislación_integral', 'provisión_servicio_seguridad', 'participación_proyecto', 'oferta_especializado_estudio_grado', 'ciberseguridad_país', 'plan_sectorial', 'comisión_mercado', 'policía_belice', 'calidad_software_control_criptográfico_seguro'
'cultura_nacional', 'subregional_protección', 'seguridad_divulgación', 'tecnología_seguridad_cibernético', 'criptográfico_seguro_cibernético', '_desfavorable', 'electrónico_definición', 'aplicación_existente', 'elaborado_forma', 'ámbito_internacional', 'consciente_importancia', '_the_', 'ministerio', 'ciudad_autónomo', 'nacional_gobierno'
               ]

for element in curated_list:
    for i in sort_orders_terms:
        if element in i[0] or element == i[0]:
            print(i[0], i[1])
            terms_to_remove.append(i[0])


In [ ]:
from nltk.corpus import stopwords
import spacy, en_core_web_lg
nlp_en = spacy.load('en_core_web_lg', disable=['ner', 'tagger', 'parser'])

stop_words_es = stopwords.words('spanish')
stop_words_en = stopwords.words('english')

final_stop_words = stop_words_es + stop_words_en

# Spacy stop_words
final_stop_words.extend(nlp_en.Defaults.stop_words)

In [ ]:
final_stop_words = final_stop_words + ['alto_efectividad', 'análisis_costo-efectividad', 'bajo_efectividad', 'bajo_efectividad_policial',  'cobertura_efectivo',  'costo_efectividad', 'costo_efectivo', 'efectividad',  'efectividad_cumplimiento', 'efectividad_gasto', 'efectividad_gestión', 'efectividad_gubernamental', 'efectividad_intervención', 'efectividad_programa', 'efectivo', 'efectivo_servicio', 'evaluación_efectividad', 'evidencia_efectividad', 'inefectivo', 'mejora_efectividad', 'política_efectivo', 'práctica_efectivo', 'índice_efectividad', 'sostenido', 'adicional', 'adicionalidad', 'agricultura_tradicional', 'autoridad_tradicional', 'medida_adicional', 'recurso_adicional', 'tradicional', 'tradicional_comercio', 'adquisición', 'acceso_mercado_internacional', 'acción_nacional', 'acuerdo_internacional', 'administración_nacional', 'aeropuerto_internacional', 'agencia_español_cooperación_internacional', 'agencia_nacional', 'agropecuario_internacional', 'análisis_experiencia_internacional', 'apoyo_internacionalización', 'asociación_nacional', 'autoridad_nacional', 'autoridad_nacional_aduana', 'banco_nacional', 'banco_nacional_desarrollo', 'binacional', 'binacional_salto', 'binacional_salto_grande', 'catálogo_nacional', 'censo_nacional', 'censo_nacional_agropecuario', 'censo_nacional_población', 'comercio_internacional', 'comisión_nacional', 'comité_nacional', 'compromiso_internacional', 'conferencia_internacional', 'consejo_nacional', 'consejo_nacional_competitividad', 'contribución_determinado_nivel_nacional', 'cooperación_internacional', 'cooperación_internacional_desarrollo', 'coordinación_binacional', 'coordinador_nacional', 'corporación_nacional', 'creación_sistema_nacional', 'demanda_turístico_internacional', 'departamento_nacional', 'desarrollo_estrategia_nacional', 'desarrollo_nacional', 'determinado_nivel_nacional', 'digital_nacional', 'dirección_nacional', 'diálogo_experto_internacional', 'diálogo_nacional', 'eléctrico_nacional', 'emergencia_nacional', 'empresa_multinacional', 'encuesta_nacional', 'encuesta_nacional_salud', 'encuesta_nacional_salud_sexual', 'energético_nacional', 'ente_subnacional', 'entidad_nacional', 'entidad_subnacional', 'español_cooperación_internacional', 'estadístico_nacional', 'estrategia_digital_nacional', 'estrategia_nacional', 'estrategia_nacional_desarrollo', 'estrategia_nacional_desarrollo_territorial', 'estrategia_nacional_innovación', 'estrategia_nacional_política', 'estrategia_nacional_política_público', 'estrategia_nacional_prevención', 'estrategia_nacional_redd+', 'estratégico_nacional', 'estructuración_túnel_internacional', 'estándar_internacional', 'evaluación_nacional', 'evento_internacional', 'evidencia_internacional', 'experiencia_internacional', 'experto_internacional', 'financiero_internacional', 'financiero_nacional', 'fiscal_subnacional', 'fondo_nacional', 'fortalecimiento_sistema_nacional', 'geológico_nacional', 'gestión_fiscal_subnacional', 'gobierno_nacional', 'gobierno_sub-nacional', 'gobierno_subnacional', 'implementación_estrategia_nacional', 'implementación_política_nacional', 'información_nacional', 'infraestructura_nacional', 'inserción_internacional', 'institución_nacional', 'instituto_nacional', 'instituto_nacional_estadístico', 'instituto_nacional_mujer', 'internacional_dato', 'internacional_desarrollo', 'internacional_protección', 'internacional_sector', 'internacional_seguridad', 'internacionalización_empresa', 'internacionalización_sector', 'inventario_nacional', 'inversión_nacional', 'laboratorio_nacional', 'laboratorio_nacional_agropecuario', 'laboratorio_nacional_referencia', 'ley_nacional', 'licitación_internacional', 'licitación_público_internacional', 'llegado_internacional', 'marco_nacional', 'media_nacional', 'mejor_práctica_internacional', 'mejoramiento_sistema_nacional', 'mercado_internacional', 'monetario_internacional', 'multinacional', 'nacional_acción', 'nacional_aduana', 'nacional_agropecuario', 'nacional_agua', 'nacional_alerta', 'nacional_alerta_temprano', 'nacional_ambiente', 'nacional_argentino', 'nacional_beca', 'nacional_cafetalero', 'nacional_calidad', 'nacional_cambio', 'nacional_cambio_climático', 'nacional_ciencia', 'nacional_competitividad', 'nacional_control', 'nacional_desarrollo', 'nacional_desarrollo_productivo', 'nacional_desarrollo_territorial', 'nacional_discapacidad', 'nacional_distribución', 'nacional_ee', 'nacional_eficiencia', 'nacional_eficiencia_energético', 'nacional_electricidad', 'nacional_energía', 'nacional_estadística', 'nacional_estadístico', 'nacional_evaluación', 'nacional_fomento', 'nacional_formación', 'nacional_gestión', 'nacional_hogar', 'nacional_identificación', 'nacional_información', 'nacional_innovación', 'nacional_integrado', 'nacional_integrado_salud', 'nacional_inversión', 'nacional_inversión_público', 'nacional_investigación', 'nacional_logística', 'nacional_mipyme', 'nacional_monitoreo', 'nacional_mujer', 'nacional_ordenamiento', 'nacional_ordenamiento_territorial', 'nacional_panamá', 'nacional_planeación', 'nacional_población', 'nacional_política', 'nacional_política_público', 'nacional_prevención', 'nacional_productividad', 'nacional_redd+', 'nacional_reducción', 'nacional_referencia', 'nacional_regulación', 'nacional_riesgo', 'nacional_salud', 'nacional_salud_sexual', 'nacional_seguridad', 'nacional_seguridad_público', 'nacional_seguridad_vial', 'nacional_servicio', 'nacional_tecnología', 'nacional_tecnología_información', 'nacional_transmisión', 'nacional_uso_eficiente', 'nacionalidad', 'nivel_internacional', 'nivel_nacional', 'nivel_sub-nacional', 'nivel_subnacional', 'norma_internacional', 'norma_nacional', 'observatorio_nacional', 'oficina_nacional', 'oficina_nacional_estadístico', 'orden_nacional', 'organismo_internacional', 'organización_internacional', 'parque_nacional', 'pib_nacional', 'plan_acción_nacional', 'plan_estratégico_nacional', 'plan_nacional', 'plan_nacional_acción', 'plan_nacional_desarrollo', 'plan_nacional_eficiencia', 'plan_nacional_eficiencia_energético', 'plan_nacional_seguridad', 'planificación_nacional', 'plurinacional', 'plurinacional_bolivia', 'policía_nacional', 'política_energético_nacional', 'política_nacional', 'política_nacional_gestión', 'política_nacional_seguridad', 'política_público_nacional', 'posicionamiento_internacional', 'precio_internacional', 'presupuesto_nacional', 'prioridad_nacional', 'programa_estructuración_túnel_internacional', 'programa_nacional', 'programa_nacional_beca', 'promedio_nacional', 'promoción_internacional', 'prueba_internacional', 'prueba_nacional', 'práctica_internacional', 'público_internacional', 'público_nacional', 'red_nacional', 'red_vial_nacional', 'refuerzo_nacional', 'registro_nacional', 'reporte_nacional', 'reserva_internacional', 'ruta_nacional', 'sat_nacional', 'secretaría_nacional', 'sector_público_nacional', 'seminario_internacional', 'servicio_nacional', 'sistema_estadístico_nacional', 'sistema_financiero_nacional', 'sistema_información_nacional', 'sistema_nacional', 'sistema_nacional_alerta', 'sistema_nacional_alerta_temprano', 'sistema_nacional_control', 'sistema_nacional_distribución', 'sistema_nacional_identificación', 'sistema_nacional_información', 'sistema_nacional_innovación', 'sistema_nacional_integrado', 'sistema_nacional_integrado_salud', 'sistema_nacional_inversión', 'sistema_nacional_inversión_público', 'sistema_subnacional', 'sistema_vigilancia_agropecuario_internacional', 'subnacional', 'superior_media_nacional', 'supranacional', 'taller_diálogo_experto_internacional', 'taller_internacional', 'territorio_nacional', 'transnacional', 'trinacional', 'tránsito_internacional', 'turismo_internacional', 'turístico_internacional', 'túnel_internacional', 'universidad_nacional', 'vial_nacional', 'vigilancia_agropecuario_internacional', 'visitante_internacional', 'índice_nacional', 'apoyo_específico', 'asociado_problema_específico', 'capacitación_específico', 'causa_asociado_problema_específico', 'específico_producto', 'estudio_específico', 'generación_instrumento_específico', 'instrumento_específico', 'inversión_específico', 'objetivo_específico', 'país_específico', 'problema_específico', 'proyecto_específico', 'reembolsable_proyecto_específico', 'funcionamiento', 'funcionamiento_mercado', 'actividad_principal', 'causa_principal', 'colector_principal', 'función_principal', 'identificación_principal', 'objetivo_principal', 'principal_actividad', 'principal_actor', 'principal_barrera', 'principal_beneficiario', 'principal_causa', 'principal_ciudad', 'principal_corredor', 'principal_corredor_urbano', 'principal_desafío', 'principal_determinante', 'principal_factor', 'principal_factor_determinante', 'principal_fuente', 'principal_lección', 'principal_lección_aprendido', 'principal_medida', 'principal_mercado', 'principal_objetivo', 'principal_país', 'principal_problema', 'principal_problemática', 'principal_proceso', 'principal_producto', 'principal_programa', 'principal_resultado', 'principal_riesgo', 'principal_rubro', 'principal_socio', 'principal_socio_comercial', 'problema_principal', 'producto_principal', 'red_principal', 'subcomponente', 'subcomponentes', 'desperdiciado', 'desperdicio', 'espera', 'esperable', \
                    'esperado', 'esperanza', 'gestión_lista_espera', 'gestión_tiempo_espera', 'inesperado', 'alcance_resultado', 'análisis_resultado', 'basado_resultado', 'clave_resultado', 'comunicación_resultado', 'difusión_resultado', 'diseminación_resultado', 'enfoque_resultado', 'evaluación_resultado', 'financiamiento_basado_resultado', 'focalizado_resultado', 'gestión_resultado', 'indicador_clave_resultado', 'indicador_resultado', 'marco_resultado', 'marco_resultado_corporativo', 'matriz_resultado', 'mejor_resultado', 'modelo_financiamiento_basado_resultado', 'monitoreo_resultado', 'plan_comunicación_resultado', 'presentación_resultado', 'principal_resultado', 'result', 'resultado_actividad', 'resultado_alcanzado', 'resultado_análisis', 'resultado_aprendizaje', 'resultado_corporativo', 'resultado_ct', 'resultado_educativo', 'resultado_encuesta', 'resultado_estudio', 'resultado_estudio_gobierno', 'resultado_evaluación', 'resultado_fiscal', 'resultado_institucional', 'resultado_intermedio', 'resultado_intervención', 'resultado_obtenido', 'resultado_piloto', 'resultado_positivo', 'resultado_previsto', 'resultado_primario', 'resultado_programa', 'resultado_proyecto', 'resultado_prueba', 'resultado_salud', 'resultado_sanitario', 'resultante', 'capacidad_agencia', 'coordinación_agencia', 'institucional_agencia', 'agencia_cooperación', 'agencia_desarrollo', 'agencia_español', 'agencia_español_cooperación', 'agencia_español_cooperación_internacional', 'agencia_francés', 'agencia_francés_desarrollo', 'agencia_gobierno', 'agencia_gubernamental', 'agencia_gubernamental_beneficiado', 'agencia_gubernamental_beneficiado_proyecto', 'agencia_innovación', 'agencia_investigación', 'agencia_mujer', 'agencia_nacional', 'agencia_promoción', 'agencia_promoción_exportación', 'agencia_promoción_inversión', 'agencia_público', 'agencia_público_innovación', 'agencia_regulación', 'agencia_regulador', 'operación_cooperación','accidente_tránsito_américa_latino', 'agua_subterráneo_américa_latino', 'américa_latino', 'américa_latino_nuevo_ruta', 'arroz_américa_latino', 'arroz_américa_latino_flar', 'banco_desarrollo_américa_latino', 'bono_verde_américa_latino', 'ciudad_américa_latino', 'desarrollo_américa_latino', 'económico_américa_latino', 'educativo_américa_latino', 'estadística_tributario_américa_latino', 'gobierno_américa_latino', 'gobierno_subnacional_américa_latino', 'infraestructura_económico_américa_latino', 'integración_américa_latino', 'movilidad_américa_latino', 'observatorio_arroz_américa_latino', 'país_américa_latino', 'pensión_américa_latino', 'política_américa_latino', 'proceso_integración_américa_latino', 'promedio_américa_latino', 'práctica_simus_américa_latino', 'región_américa_latino', 'saneamiento_américa_latino', 'sector_agropecuario_américa_latino', 'sistema_educativo_américa_latino', 'subnacional_américa_latino', 'subterráneo_américa_latino', 'tributario_américa_latino', 'tránsito_américa_latino', 'verde_américa_latino', 'américa_latino', 'país_américa_latino', 'política_américa_latino', 'promedio_américa_latino', 'región_américa_latino', 'fortalecimiento_organismo', 'institucional_organismo', 'organismo', 'organismo_control', 'organismo_encargado', 'organismo_gubernamental', 'organismo_internacional', 'organismo_multilateral', 'organismo_privado', 'organismo_público', 'organismo_técnico', 'actividad/_componente_descripción', 'actividad/_componente_descripción_bid', 'actividad/_componente_descripción_bid_financiamiento', 'componente_descripción', 'componente_descripción_actividad', 'componente_descripción_actividad_bid', 'componente_descripción_bid', 'componente_descripción_bid/', 'componente_descripción_bid_contrapartida', 'componente_descripción_bid_contrapartida_local', 'componente_descripción_bid_contrapartida_local_financiamie', 'componente_descripción_bid_financiamiento', 'componente_descripción_bid_financiamiento_fondo', 'componente_descripción_bid_financiamientototal', 'componente_descripción_bid_local', 'componente_descripción_bid_total_componente', 'componente_descripción_financiamiento', 'componente_descripción_financiamiento_bid', 'componente_descripción_financiamiento_financiamiento', 'componente_descripción_financiamiento_financiamiento_total_componente', 'componente_descripción_financiamiento_fondo', 'estructura_ejecución', 'actualización_estrategia_institucional', 'estrategia_institucional', 'estrategia_institucional_anii', 'estrategia_institucional_banco',  'implementación_estrategia_institucional', 'área_transversal', 'área_transversal_cambio', 'área_transversal_cambio_climático', 'análisis_capacidad_institucional', 'bajo_capacidad_institucional', 'capacidad_institucional', 'capacidad_institucional_agencia', 'capacidad_institucional_anii', 'capacidad_institucional_autoridad', 'capacidad_institucional_beneficiario', 'capacidad_institucional_bfa', 'capacidad_institucional_der', 'capacidad_institucional_dpc', 'capacidad_institucional_finep', 'capacidad_institucional_gestión', 'capacidad_institucional_gobierno', 'capacidad_institucional_implementación', 'capacidad_institucional_jamaica', 'capacidad_institucional_medición', 'capacidad_institucional_ministerio', 'capacidad_institucional_ministerio_finanzas', 'capacidad_institucional_ministerio_salud', 'capacidad_institucional_municipio', 'capacidad_institucional_municipio_brasileño', 'capacidad_institucional_organismo', 'capacidad_institucional_prevención', 'capacidad_institucional_sector', 'capacidad_institucional_sector_público', 'capacidad_institucional_ssa', 'capacidad_institucional_sunafil', 'construcción_capacidad_institucional', 'creación_capacidad_institucional', 'desarrollo_capacidad_institucional', 'diagnóstico_capacidad_institucional', 'división_capacidad_institucional', 'débil_capacidad_institucional', 'evaluación_capacidad_institucional', 'fortalecimiento_capacidad_institucional', 'fortalecimiento_capacidad_institucional_sector', 'limitado_capacidad_institucional', 'mejora_capacidad_institucional', 'tema_transversal_capacidad_institucional', 'transversal_capacidad_institucional', 'acceso_servicio_asistencia_técnico', 'actividad_asistencia_técnico', 'asistencia_técnico', 'asistencia_técnico_a', 'asistencia_técnico_agricultor', 'asistencia_técnico_bfp', 'asistencia_técnico_complementario', 'asistencia_técnico_complementario_implementación', 'asistencia_técnico_continuo', 'asistencia_técnico_continuo_a', 'asistencia_técnico_crediticia', 'asistencia_técnico_definición', 'asistencia_técnico_desarrollo', 'asistencia_técnico_diseño', 'asistencia_técnico_elaboración', 'asistencia_técnico_especialista', 'asistencia_técnico_especializado', 'asistencia_técnico_estudio', 'asistencia_técnico_estudio_complementario', 'asistencia_técnico_experto', 'asistencia_técnico_experto_internacional', 'asistencia_técnico_fortalecimiento', 'asistencia_técnico_gestión', 'asistencia_técnico_gestión_proyecto', 'asistencia_técnico_idaan', 'asistencia_técnico_implementación', 'asistencia_técnico_implementación_política', 'asistencia_técnico_inam', 'asistencia_técnico_institución', 'asistencia_técnico_institución_cdg', 'asistencia_técnico_institución_educativo', 'asistencia_técnico_integral', 'asistencia_técnico_jass', 'asistencia_técnico_mejoramiento', 'asistencia_técnico_ministerio', 'asistencia_técnico_ministerio_sectorial', 'asistencia_técnico_mipyme', 'asistencia_técnico_operativo', 'asistencia_técnico_organización', 'asistencia_técnico_organización_local', 'asistencia_técnico_país', 'asistencia_técnico_productor', 'asistencia_técnico_requerido', 'asistencia_técnico_requerido_ejecución', 'asistencia_técnico_resultado', 'asistencia_técnico_revisión', 'asistencia_técnico_sam', 'asistencia_técnico_seguridad', 'asistencia_técnico_seguridad_cibernético', 'asistencia_técnico_sistema', 'asistencia_técnico_transferencia', 'asistencia_técnico_uamp', 'asistencia_técnico_virtual', 'consultoría_asistencia_técnico', 'contratación_asistencia_técnico', 'contratación_asistencia_técnico_diseño', 'empresa_asistencia_técnico', 'especializado_asistencia_técnico', 'especializado_asistencia_técnico_gestión', 'especializado_asistencia_técnico_requerido', 'estrategia_asistencia_técnico', 'financiamiento_asistencia_técnico', 'firma_especializado_asistencia_técnico', 'fortalecimiento_área_asistencia_técnico', 'importancia_asistencia_técnico', 'local_asistencia_técnico', 'local_servicio_asistencia_técnico', 'prestación_asistencia_técnico', 'prestación_asistencia_técnico_a', 'prestador_asistencia_técnico', 'prestador_local_asistencia_técnico', 'proceso_asistencia_técnico', 'programa_asistencia_técnico', 'provisión_asistencia_técnico', 'provisión_asistencia_técnico_a', 'provisión_asistencia_técnico_complementario', 'provisión_asistencia_técnico_organización', 'proyecto_asistencia_técnico', 'servicio_asistencia_técnico', 'servicio_especializado_asistencia_técnico', 'servicio_especializado_asistencia_técnico_requerido', 'área_asistencia_técnico', 'área_asistencia_técnico_bfp', 'único', 'consiguiente', 'plazo', 'corto_plazo', 'mediano_plazo','largo_plazo', 'subsiguiente', 'cooperación', 'actividad_complementario_cooperación_técnico', 'actividad_cooperación_técnico', 'complementario_cooperación_técnico', \
                    'cooperación_técnico', 'cooperación_técnico_atn', 'cooperación_técnico_fortalecimiento', 'cooperación_técnico_incluido', 'cooperación_técnico_individual', 'cooperación_técnico_no-reembolsable', 'cooperación_técnico_regional', 'financiado_presente_cooperación_técnico', 'financiamiento_cooperación_técnico', 'operación_cooperación_técnico', 'operación_cooperación_técnico_atn', 'operación_cooperación_técnico_fortalecimiento', 'operación_cooperación_técnico_incluido', 'operación_cooperación_técnico_individual', 'operación_cooperación_técnico_regional', 'presente_cooperación_técnico', 'presente_proyecto_cooperación_técnico', 'proyecto_cooperación_técnico', 'préstamo_cooperación_técnico', 'recurso_cooperación_técnico', '▪_operación_cooperación_técnico', '▪_operación_cooperación_técnico_incluido', 'operación_préstamo', 'préstamo_inversión', 'préstamo_a', 'ejecución_préstamo', 'préstamo_apoyo', 'préstamo_programático', 'recurso_préstamo', 'préstamo_apoyo_reforma', 'préstamo_programático_apoyo', 'préstamo_programático_apoyo_reforma', 'préstamo_banco', 'préstamo_mipyme', 'preparación_operación_préstamo', 'preparación_préstamo', 'programa_préstamo', 'préstamo_bid', 'préstamo_asociado', 'préstamo_garantía_soberano', 'préstamo_garantía', 'cartera_préstamo', 'evaluación_impacto_préstamo', 'impacto_préstamo', 'contrato_préstamo', 'serie_préstamo', 'serie_préstamo_programático_apoyo', 'serie_préstamo_programático', 'individual_préstamo', 'operación_individual_préstamo', 'solicitud_préstamo', 'préstamo_pyme', 'monto_préstamo', 'préstamo_personal', 'periodo_desembolso_préstamo', 'desembolso_préstamo', 'préstamo_cooperación', 'préstamo_cooperación_técnico', 'género_préstamo', 'préstamo_co-l1229', 'préstamo_recibido', 'préstamo_pactado', 'préstamo_reforma', 'préstamo_reforma_política', 'apoyo_préstamo', 'préstamo_garantizado', 'préstamo_comercial', 'préstamo_inversión_específico', 'inicio_ejecución_préstamo', 'estrategia_género_préstamo', 'género_préstamo_bid', 'implementado_préstamo', 'préstamo_bo-l1111', 'implementación_operación_préstamo', 'préstamo_br-l1503', 'operación_préstamo_br-l1503', 'préstamo_ec-l1071', 'seguimiento_préstamo', 'preparación_préstamo_inversión', 'estudio_preparación_préstamo', 'proyecto_préstamo_me-l', 'préstamo_me-l', 'proyecto_préstamo', 'supervisión_préstamo', 'monitoreo_préstamo', 'ejecución_operación_préstamo', 'organismo_ejecutor_préstamo', 'ejecutor_préstamo', 'apoyo_preparación_préstamo', 'préstamo_plan_inversión', 'préstamo_plan', 'preparación_préstamo_plan', 'préstamo_preparación', 'intervención_préstamo', 'intervención_préstamo_co-l1229', 'diseño_intervención_préstamo', 'operación_préstamo_inversión', 'oferta_recurso_préstamo', 'préstamo_inversión_productivo', 'duración_préstamo_mediano', 'duración_préstamo_a', 'duración_préstamo', 'préstamo_mediano', 'préstamo_recibido_banco', 'desarrollado_primero_préstamo', 'préstamo_global', 'recurso_proveniente_préstamo', 'proveniente_préstamo', 'contratación_préstamo', 'préstamo_original', 'aprobación_préstamo', 'préstamo_empresarial', 'préstamo_banco_público', 'concesión_préstamo', 'diseño_operación_préstamo', 'bfa_préstamo', 'préstamo_familiar', 'cartera_bfa_préstamo', 'préstamo_largo_plazo', 'préstamo_largo_plazo_a', 'bfa_préstamo_largo_plazo', 'préstamo_banco_comercial', 'préstamo_garantizar', 'implementación_préstamo', 'préstamo_senior_a', 'préstamo_senior', 'financiamiento_préstamo_inversión', 'financiamiento_préstamo', 'préstamo_especial', 'préstamo_especial_desarrollo', 'préstamo_inversión_gomo', 'fondo_garantía_préstamo', 'fondo_garantía_préstamo_a', 'garantía_préstamo_a', 'garantía_préstamo', 'préstamo_apoyo_a', 'alimento_organización', 'apoyo_organización', 'apoyo_organización_encuentro', 'apoyo_reorganización', 'apoyo_reorganización_funcional', 'asistencia_técnico_organización', 'asistencia_técnico_organización_local', 'brecha_organización', 'capacitación_organización', 'colaboración_organización', 'colaboración_organización_nacional', 'construcción_red_organización', 'coordinación_organización', 'coordinación_organización_derecho', 'empoderamiento_organización', 'evaluación_operativo_reorganización', 'fortalecimiento_organización', 'fortalecimiento_organización_intermediario', 'fortalecimiento_organización_intermediario_transferencia', 'institucionalización_nuevo_organización', 'logística_relacionado_organización', 'mejora_organización', 'mejora_organización_institucional', 'modelo_organización', 'operativo_organización', 'operativo_reorganización', 'operativo_reorganización_mid', 'optimización_operativo_organización', 'organización_actividad', 'organización_actividad_promoción', 'organización_actual', 'organización_agricultor', 'organización_agricultor_familiar', 'organización_agricultura', 'organización_agricultura_familiar', 'organización_aliado', 'organización_asociado', 'organización_base', 'organización_capaz', 'organización_civil', 'organización_civil_social', 'organización_climate', 'organización_climate_bonds', 'organización_climate_bonds_initiative', 'organización_comunitario', 'organización_comunitario_asociado', 'organización_comunitario_servicio', 'organización_comunitario_servicio_agua', 'organización_concurso', 'organización_cooperación', 'organización_derecho', 'organización_desarrollo', 'organización_ecosistema', 'organización_encuentro', 'organización_encuentro_empresarial', 'organización_evento', 'organización_forestal', 'organización_foro', 'organización_funcional', 'organización_indígena', 'organización_indígena_personería', 'organización_indígena_personería_jurídico', 'organización_institucional', 'organización_intermediario', 'organización_intermediario_transferencia', 'organización_intermediario_transferencia_tecnológico', 'organización_internacional', 'organización_internacional_dedicado', 'organización_internacional_dedicado_financiamiento', 'organización_interno', 'organización_local', 'organización_meteorológico', 'organización_meteorológico_mundial', 'organización_mujer', 'organización_mundial', 'organización_mundial_aduana', 'organización_mundial_comercio', 'organización_mundial_salud', 'organización_mundial_sanidad', 'organización_mundial_sanidad_animal', 'organización_nacional', 'organización_panamericano', 'organización_panamericano_salud', 'organización_productor', 'organización_promotor', 'organización_promotor_comercio', 'organización_regante', 'organización_rural', 'organización_seminario', 'organización_seminario_internacional', 'organización_servicio', 'organización_servicio_atención', 'organización_social', 'organización_sociedad', 'organización_sociedad_civil', 'organización_taller', 'organización_taller_actor', 'principio_organización', 'principio_organización_servicio', 'provisión_asistencia_técnico_organización', 'red_organización', 'red_organización_civil', 'red_organización_civil_social', 'relacionado_organización', 'reorganización_funcional', 'reorganización_institucional', 'reorganización_institucional_mid', 'reorganización_mid', 'reorganización_servicio', 'revisión_organización', 'servicio_organización', 'servicio_organización_comunitario', 'servicio_organización_comunitario_asociado', 'técnica_organización', 'técnica_organización_local', 'actividad_privado_afectado_emergencia', 'afectado',  'afectado_causa', 'afectado_conflicto', 'afectado_conflicto_armado', 'afectado_construcción', 'afectado_crisis', 'afectado_crisis_covid-19', 'afectado_crisis_generado', 'afectado_emergencia', 'afectado_emergencia_nacional', 'afectado_emergencia_nacional_sanitario', 'afectado_estrés', 'afectado_estrés_hídrico', 'afectado_fenómeno', 'afectado_homicidio', 'afectado_huracán_dorian', 'afectado_huracán_joaquín', 'afectado_huracán_matthew', 'afectado_pandemia', 'afectado_plaga', 'afectado_reformulación', 'afectado_sismo', 'afectado_terremoto', 'clave_afectado_covid-19', 'comunidad_afectado_construcción', 'eed_afectado_terremoto', 'mipe_afectado_crisis', 'mipyme_afectado_crisis_generado', 'persona_afectado_causa', 'perú_afectado_fenómeno', 'privado_afectado_emergencia', 'privado_afectado_emergencia_nacional', 'pyme_afectado_crisis', 'rural_afectado_huracán', 'rural_afectado_huracán_matthew', 'zona_afectado_sismo', 'zona_rural_afectado_huracán', 'área_clave_afectado_covid-19', 'aaf', 'aao', 'aaps', 'órgano_administración', 'único_conjunto', 'único_conjunto_sistema', 'único_permiso', 'único_permiso_acceso', 'único_permiso_acceso_especial', 'acceso_servicio_administración', 'actor_clave_administración', 'actualización_sistema_administración', 'administración_activo', 'administración_aeropuerto', 'administración_agrícola', 'administración_caso', 'administración_caso_ii', 'administración_central', 'administración_contrato', 'administración_directo', 'administración_educación', 'administración_escolar', 'administración_estado_peruano', 'administración_fiduciario', 'administración_financiero', 'administración_financiero_país', 'administración_financiero_público', 'administración_financiero_sector', 'administración_financiero_sector_público', 'administración_fiscal', 'administración_fondo', 'administración_gasto', 'administración_gasto_público', 'administración_impuesto', 'administración_impuesto_interno', \
                    'administración_impuesto_predial', 'administración_indirecto', 'administración_ingreso', 'administración_ingreso_fiscal', 'administración_integral', 'administración_integral_contrato', 'administración_justicia', 'administración_justicia_penal', 'administración_justicia_penal_a', 'administración_línea', 'administración_línea_seguimiento', 'administración_mantenimiento', 'administración_mecanismo', 'administración_nacional', 'administración_nacional_electricidad', 'administración_pasivo', 'administración_penitenciario', 'administración_pf', 'administración_plataforma', 'administración_presupuestaria', 'administración_programa', 'administración_proyecto', 'administración_puente', 'administración_puerto', 'administración_público', 'administración_público_a', 'administración_público_federal', 'administración_público_ineficiente', 'administración_público_local', 'administración_público_nacional', 'administración_regional', 'administración_riesgo', 'administración_riesgo_ambiental', 'administración_servicio', 'administración_servicio_salud', 'administración_snd', 'administración_tierra', 'administración_tierra_rural', 'administración_tributario', 'administración_tributario_alc', 'administración_tributario_belize', 'administración_tributario_impuesto', 'administración_tributario_impuesto_predial', 'administración_tributario_integrado', 'administración_tributario_jamaica', 'ascenso_administración', 'capacidad_administración', 'capacitación_administración', 'clave_administración', 'comisión_administración', 'compañía_administración', 'comunicación_administración', 'costo_administración', 'creación_manual_gobernanza_administración', 'cuota_administración', 'departamento_administración', 'digital_administración', 'digitalización_administración', 'diseño_macroproceso_administración', 'división_administración', 'dotación_personal_administración', 'eficiencia_administración', 'especializado_sistema_administración', 'estrategia_implementación_administración', 'financiero_administración', 'fortalecimiento_administración', 'funcionario_administración', 'función_administración', 'gasto_administración', 'gestión_administración', 'gobernanza_administración', 'humanos_administración', 'i._administración', 'implementación_administración', 'implementación_nuevo_sistema_administración', 'información_administración', 'instalación_administración', 'integrado_administración', 'intercambio_funcionario_administración', 'interoperabilidad_sistema_administración', 'macroproceso_administración', 'manual_gobernanza_administración', 'marco_normativo_nuevo_administración', 'mejora_administración', 'modelo_administración', 'modernización_sistema_administración', 'modernización_tecnológico_administración', 'nivel_administración', 'normativo_nuevo_administración', 'nómina_administración', 'oficina_administración', 'operación_administración', 'operativo_administración', 'oportunidad_ascenso_administración', 'organismo_administración', 'personal_administración', 'proceso_administración', 'recurso_humano_administración', 'reforma_digital_administración', 'revisión_proceso_administración', 'servicio_administración', 'sistema_administración', 'sistema_integrado_administración', 'solución_tecnológico_administración', 'tecnológico_administración', 'tic_administración', 'transformación_digital_administración', 'unidad_administración', 'órgano_administración', 'actividad_enfoque', 'aplicación_enfoque', 'apoyo_a', 'apoyo_acción', 'apoyo_acción_política', 'apoyo_actividad', 'apoyo_actividad_difusión', 'apoyo_adaptación', 'apoyo_adopción', 'apoyo_adopción_esquema', 'apoyo_adopción_niif', 'apoyo_adquisición', 'apoyo_agenda', 'apoyo_agenda_evaluación', 'apoyo_agenda_integración', 'apoyo_agenda_modernización', 'apoyo_análisis', 'apoyo_análisis_estadístico', 'apoyo_apertura', 'apoyo_apertura_dato', 'apoyo_aplicación', 'apoyo_aplicación_conocimiento', 'apoyo_aplicación_tecnología', 'apoyo_aprovechamiento', 'apoyo_aprovechamiento_sostenible', 'apoyo_armado', 'apoyo_armado_institucional', 'apoyo_arranque', 'apoyo_asistencial', 'apoyo_atención', 'apoyo_atención_persona', 'apoyo_banco', 'apoyo_banco_a', 'apoyo_bandesal', 'apoyo_bdc', 'apoyo_bid', 'apoyo_cambio', 'apoyo_campaña', 'apoyo_campaña_seguridad', 'apoyo_campaña_seguridad_vial', 'apoyo_capacidad', 'apoyo_caracterización', 'apoyo_censo', 'apoyo_cgr', 'apoyo_ciudad', 'apoyo_cobranza', 'apoyo_comisión', 'apoyo_comisión_protección', 'apoyo_competitividad', 'apoyo_complementario', 'apoyo_componente', 'apoyo_compromiso', 'apoyo_comunidad', 'apoyo_comunitario', 'apoyo_consejo', 'apoyo_consejo_ministro', 'apoyo_conservación', 'apoyo_consolidación', 'apoyo_consultor', 'apoyo_consultor_individual', 'apoyo_consultor_individual_dedicado', 'apoyo_consultor_individual_dedicado_a', 'apoyo_consultoría', 'apoyo_consultoría_especializado', 'apoyo_contraparte', 'apoyo_coordinación', 'apoyo_coordinación_agencia', 'apoyo_coordinación_interinstitucional', 'apoyo_creación', 'apoyo_creación_ministerio', 'apoyo_cte-mrpp', 'apoyo_cti&e', 'apoyo_cuidado', 'apoyo_cuidado_persona', 'apoyo_cumplimiento', 'apoyo_cumplimiento_condición', 'apoyo_dedicado', 'apoyo_dedicado_a', 'apoyo_dedicado_subregión', 'apoyo_defensa', 'apoyo_defensa_tejido', 'apoyo_definición', 'apoyo_desarrollo', 'apoyo_desarrollo_actividad', 'apoyo_desarrollo_actividad_dirigido', 'apoyo_desarrollo_capacidad', 'apoyo_desarrollo_empresa', 'apoyo_desarrollo_empresa_digital', 'apoyo_desarrollo_estudio', 'apoyo_desarrollo_estudio_estratégico', 'apoyo_desarrollo_iniciativa', 'apoyo_desarrollo_modelo', 'apoyo_desarrollo_perfil', 'apoyo_desarrollo_plan', 'apoyo_desarrollo_sistema', 'apoyo_desarrollo_tecnología', 'apoyo_desarrollo_urbano', 'apoyo_descarbonización', 'apoyo_determinación', 'apoyo_determinación_índice', 'apoyo_diagnóstico', 'apoyo_diagnóstico_estado_nutricional', 'apoyo_difusión', 'apoyo_directo', 'apoyo_diseño', 'apoyo_diseño_estrategia', 'apoyo_diseño_estudio', 'apoyo_diseño_intervención', 'apoyo_diseño_inversión', 'apoyo_diseño_sistema', 'apoyo_diálogo', 'apoyo_diálogo_local', 'apoyo_diálogo_nacional', 'apoyo_diálogo_técnico', 'apoyo_docente', 'apoyo_económico', 'apoyo_educativo', 'apoyo_ee', 'apoyo_ejecución', 'apoyo_ejecución_programa', 'apoyo_ejecución_proyecto', 'apoyo_elaboración', 'apoyo_em', 'apoyo_empleo_ii', 'apoyo_emprendedor', 'apoyo_emprendimiento', 'apoyo_empresa', 'apoyo_empresa_turismo', 'apoyo_empresarial', 'apoyo_enfoque', 'apoyo_enfoque_red', 'apoyo_enra', 'apoyo_equipo', 'apoyo_equipo_técnico', 'apoyo_ernc', 'apoyo_espacio', 'apoyo_espacio_discusión', 'apoyo_espacio_discusión_técnico', 'apoyo_específico', 'apoyo_específico_a', 'apoyo_estrategia', 'apoyo_estrategia_carbono', 'apoyo_estrategia_información', 'apoyo_estratégico', 'apoyo_estratégico_consolidación', 'apoyo_estructuración', 'apoyo_estructuración_financiero', 'apoyo_estructuración_marco', 'apoyo_estudio', 'apoyo_estudio_pre-inversión', 'apoyo_estudio_técnico', 'apoyo_evaluación', 'apoyo_evaluación_impacto', 'apoyo_evaluación_intervención', 'apoyo_exploración', 'apoyo_exploración_mejor', 'apoyo_externo', 'apoyo_fide', 'apoyo_financiamiento', 'apoyo_financiamiento_infraestructura', 'apoyo_financiamiento_infraestructura_productivo', 'apoyo_financiero', 'apoyo_financiero_banco', 'apoyo_financiero_recibido', 'apoyo_firma', 'apoyo_focalizado', 'apoyo_fomento', 'apoyo_fomento_ecosistema', 'apoyo_fondo', 'apoyo_formación', 'apoyo_formación_cha', 'apoyo_formación_destreza', 'apoyo_formulación', 'apoyo_formulación_plan', 'apoyo_fortalecimiento', 'apoyo_fortalecimiento_capacidad', 'apoyo_fortalecimiento_ecosistema', 'apoyo_fortalecimiento_gobernanza', 'apoyo_fortalecimiento_institucional', 'apoyo_fortalecimiento_manejo', 'apoyo_fortalecimiento_manejo_forestal', 'apoyo_fortalecimiento_marco', 'apoyo_fortalecimiento_marco_legal', 'apoyo_fortalecimiento_pgn', 'apoyo_fortalecimiento_sector', 'apoyo_fortalecimiento_sector_transporte', 'apoyo_fortalecimiento_sistema', 'apoyo_fortalecimiento_tema', 'apoyo_funcionamiento', 'apoyo_funcionamiento_fondo', 'apoyo_futuro', 'apoyo_futuro_empresario', 'apoyo_generación', 'apoyo_generación_conocimiento', 'apoyo_gerencial', 'apoyo_gestión', 'apoyo_gestión_adquisición', 'apoyo_gestión_ambiental', 'apoyo_gestión_conocimiento', 'apoyo_gestión_estación', 'apoyo_gestión_fondo', 'apoyo_gestión_gasto', 'apoyo_gestión_gobierno', 'apoyo_gestión_información', 'apoyo_gestión_integral', 'apoyo_gestión_pasivo', 'apoyo_gestión_permiso', 'apoyo_gestión_proyecto', 'apoyo_gestión_sanitario', 'apoyo_gestión_seguridad', 'apoyo_gestión_íntegro', 'apoyo_gobierno', 'apoyo_grupo', 'apoyo_icc', 'apoyo_identificación', 'apoyo_implementación', 'apoyo_implementación_electrificación', 'apoyo_implementación_iniciativa', 'apoyo_implementación_marco', 'apoyo_implementación_medida', 'apoyo_implementación_operación', 'apoyo_implementación_operativo', 'apoyo_implementación_pacto', 'apoyo_implementación_plan', 'apoyo_implementación_programa', 'apoyo_industria', 'apoyo_industria_creativo', 'apoyo_informático', 'apoyo_ingreso', 'apoyo_ingreso_objetivo', 'apoyo_iniciativa', 'apoyo_iniciativa_integración', 'apoyo_iniciativa_integración_regional', 'apoyo_inicio', 'apoyo_inicio_programa', \
                    'apoyo_inserción', 'apoyo_inserción_laboral', 'apoyo_institucional', 'apoyo_int', 'apoyo_integración', 'apoyo_integración_sector', 'apoyo_integral', 'apoyo_integral_a', 'apoyo_integral_plan', 'apoyo_integral_plan_nacional', 'apoyo_integral_sector', 'apoyo_integral_sector_extractivo', 'apoyo_interconexión', 'apoyo_interconexión_regional', 'apoyo_internacionalización', 'apoyo_internacionalización_sector', 'apoyo_interoperabilidad', 'apoyo_interoperabilidad_sistema', 'apoyo_inversión', 'apoyo_inversión_público', 'apoyo_investigación', 'apoyo_labor', 'apoyo_largo_plazo', 'apoyo_legal', 'apoyo_licitación', 'apoyo_lima', 'apoyo_lima_metropolitano', 'apoyo_logístico', 'apoyo_logístico_preparación', 'apoyo_mef', 'apoyo_mef_formulación', 'apoyo_mejora', 'apoyo_mejora_proceso', 'apoyo_mejoría', 'apoyo_men', 'apoyo_metodología', 'apoyo_mh', 'apoyo_mh_generación', 'apoyo_micro', 'apoyo_microfinanza', 'apoyo_mineduc', 'apoyo_ministerio', 'apoyo_ministerio_comercio', 'apoyo_mipyme', 'apoyo_mipyme_servicio', 'apoyo_mmaya', 'apoyo_modelo', 'apoyo_modelo_intervención', 'apoyo_modelo_intervención_especializado', 'apoyo_modernización', 'apoyo_monitoreo', 'apoyo_monitoreo_implementación', 'apoyo_movilidad', 'apoyo_movilidad_barrio', 'apoyo_movilidad_barrio_tacumbú', 'apoyo_mujer', 'apoyo_nación', 'apoyo_nuevo_plan', 'apoyo_nuevo_plan_estratégico', 'apoyo_nuevo_plan_estratégico_sectorial', 'apoyo_observatorio', 'apoyo_observatorio_nacional', 'apoyo_observatorio_nacional_logística', 'apoyo_obtención', 'apoyo_obtención_certificación', 'apoyo_oficina', 'apoyo_oficina_transparencia', 'apoyo_operación', 'apoyo_operador', 'apoyo_operativo', 'apoyo_operativo_a', 'apoyo_operativo_mejora', 'apoyo_operativo_seguimiento', 'apoyo_operativo_tema', 'apoyo_operativo_tema_preparación', 'apoyo_optimización', 'apoyo_optimización_operativo', 'apoyo_orepa', 'apoyo_orepa_norte', 'apoyo_organización', 'apoyo_organización_encuentro', 'apoyo_paquete', 'apoyo_paquete_productivo', 'apoyo_paquete_productivo_esquema', 'apoyo_parcial', 'apoyo_parm', 'apoyo_pedagógico', 'apoyo_pedagógico_integral', 'apoyo_pequeño', 'apoyo_pequeño_productor', 'apoyo_persona', 'apoyo_persona_discapacidad', 'apoyo_plan', 'apoyo_plan_movilidad', 'apoyo_plan_nacional', 'apoyo_planificación', 'apoyo_población', 'apoyo_población_afectado', 'apoyo_población_vulnerable', 'apoyo_política', 'apoyo_política_inserción', 'apoyo_política_público', 'apoyo_política_tributario', 'apoyo_político', 'apoyo_político_regional', 'apoyo_pre-identificación', 'apoyo_preinversión', 'apoyo_preparación', 'apoyo_preparación_fideicomiso', 'apoyo_preparación_prourbi', 'apoyo_preparación_proyecto', 'apoyo_preparación_préstamo', 'apoyo_presupuestario', 'apoyo_proceso', 'apoyo_proceso_diálogo', 'apoyo_proceso_selección', 'apoyo_programa', 'apoyo_programa_capacitación', 'apoyo_programa_propuesto', 'apoyo_programación', 'apoyo_programático', 'apoyo_programático_a', 'apoyo_programático_reforma', 'apoyo_programático_reforma_estructural', 'apoyo_proyecto', 'apoyo_préstamo', 'apoyo_psicosocial', 'apoyo_público', 'apoyo_realización', 'apoyo_realización_estudio', 'apoyo_realización_taller', 'apoyo_red', 'apoyo_reforma', 'apoyo_reforma_política', 'apoyo_regulación', 'apoyo_regularización', 'apoyo_reingeniería', 'apoyo_reingeniería_institucional', 'apoyo_reorganización', 'apoyo_reorganización_funcional', 'apoyo_revisión', 'apoyo_revisión_especificación', 'apoyo_sector', 'apoyo_sector_agropecuario', 'apoyo_seduc', 'apoyo_servicio', 'apoyo_servicio_global', 'apoyo_sistema', 'apoyo_sistema_información', 'apoyo_sistemático', 'apoyo_situación', 'apoyo_situación_emergencia', 'apoyo_situación_emergencia_sanitario', 'apoyo_sostenibilidad', 'apoyo_sostenibilidad_institucional', 'apoyo_supervisión', 'apoyo_tarea', 'apoyo_tarea_coordinación', 'apoyo_tecnológico', 'apoyo_transferencia', 'apoyo_transformación', 'apoyo_transformación_digital', 'apoyo_transformación_digital_empresarial', 'apoyo_transformación_digital_sectorial', 'apoyo_tratamiento', 'apoyo_técnico', 'apoyo_técnico_a', 'apoyo_técnico_elaboración', 'apoyo_técnico_equipo', 'apoyo_técnico_especializado', 'apoyo_técnico_especializado_tema', 'apoyo_técnico_evaluación', 'apoyo_técnico_externo', 'apoyo_técnico_gcps', 'apoyo_técnico_implementación', 'apoyo_técnico_mejora', 'apoyo_técnico_política', 'apoyo_técnico_priorización', 'apoyo_técnico_revisión', 'apoyo_técnico_seduc', 'apoyo_técnico_área', 'apoyo_técnico_área_innovación', 'apoyo_un-habitat', 'apoyo_unidad', 'apoyo_unidad_ejecutor', 'apoyo_unidad_técnico', 'apoyo_unidad_técnico_nacional', 'apoyo_vejez', 'apoyo_vejez_meta', 'apoyo_vejez_meta_fuente', 'apoyo_viceministerio', 'apoyo_viceministerio_transporte', 'apoyo_virtual', 'apoyo_área', 'apoyo_área_clave', 'calidad_gasto_enfoque', 'central_enfoque', 'ciudadanía_enfoque', 'construcción_ciudadanía_enfoque', 'degradado_área_central_enfoque', 'derivado_enfoque', 'desarrollo_enfoque', 'dpc_enfoque', 'falta_enfoque', 'gasto_enfoque', 'gestión_social_ays_enfoque', 'incorporación_enfoque', 'integración_servicio_enfoque', 'mipyme_enfoque', 'negocio_enfoque', 'oportunidad_derivado_enfoque', 'orientación_vocacional_enfoque', 'plan_negocio_enfoque', 'planificación_enfoque', 'programa_desarrollo_enfoque', 'provisión_at_enfoque', 'reproductivo_enfoque', 'salud_reproductivo_enfoque', 'servicio_apoyo_enfoque', 'servicio_enfoque', 'social_ays_enfoque', 'tecnología_enfoque', 'vocacional_enfoque', 'área_central_enfoque', 'consideración', 'consideración_cambio', 'consideración_cambio_climático', 'consideración_climático', 'consideración_género', 'consolidación', 'consolidación_conocimiento', 'consolidación_conocimiento_iniciativa', 'consolidación_fiscal', 'consolidación_información', 'consolidación_institucional', 'compilación_contribución', 'contribución_a', 'contribución_análisis', 'contribución_bid', 'contribución_bid_modernización', 'contribución_desarrollo', 'contribución_determinado', 'contribución_determinado_a', 'contribución_determinado_nivel', 'contribución_determinado_nivel_nacional', 'contribución_dimensión', 'contribución_directo', 'contribución_directo_turismo', 'contribución_dirigido', 'contribución_económico', 'contribución_especial', 'contribución_especial_mejora', 'contribución_formulación', 'contribución_formulación_política', 'contribución_indc', 'contribución_indc_a', 'contribución_innovación', 'contribución_nacional', 'contribución_nacional_determinado', 'contribución_previsto', 'contribución_ptf', 'contribución_reducción', 'contribución_sector', 'contribución_sector_a', 'contribución_€', 'monto_contribución', 'académico_nivel', 'acción_ipg_nivel', 'aceptable_nivel', 'actividad_nivel', 'actividad_promoción_nivel', 'activo_inmobiliario_nivel', 'actualización_nivel', 'adecuado_nivel', 'administrativo_nivel', 'adopción_bim_nivel', 'agua_residual_nivel', 'alianza_ciudad_sostenible_nivel', 'alimentario_nivel', 'alto_nivel', 'análisis_impacto_financiero_nivel', 'análisis_nivel', 'app_nivel', 'asignación_recurso_nivel', 'aumento_nivel', 'autonomía_nivel', 'autoridad_nivel', 'aéreo_nivel', 'bajo_nivel', 'barrio_alto_nivel', 'bim_nivel', 'café_salvadoreño_nivel', 'calidad_primero_nivel', 'cambio_nivel', 'capacidad_nivel', 'centro_excelencia_nivel', 'cgr_nivel', 'cibernético_reflejado_bajo_nivel', 'ciudad_sostenible_nivel', 'ciudadano_nivel', 'cobertura_nivel', 'comercial_sector_aéreo_nivel', 'comparado_nivel', 'competitividad_nivel', 'conducción_respuesta_nivel', 'conocimiento_alto_nivel', 'consultor_alto_nivel', 'contribución_determinado_nivel', 'coordinación_nivel', 'corporativo_nivel', 'cruce_nivel', 'currículo_formación_policial_nivel', 'dato_nivel', 'decisión_nivel', 'desarrollo_habilidad_digital_nivel', 'desarrollo_infantil_nivel', 'desigualdad_nivel', 'determinado_nivel', 'diagnóstico_nivel', 'difusión_nivel', 'digital_nivel', 'diálogo_nivel', 'docente_nivel', 'drenaje_pluvial_nivel', 'eficiencia_energético_nivel', 'eficiencia_terminal_nivel', 'elevación_nivel', 'emergencia_nivel', 'emisión_nivel', 'emprendimiento_corporativo_nivel', 'energético_nivel', 'enseñanza_nivel', 'entidad_nivel', 'entomológico_nivel', 'er_nivel', 'espacio_diálogo_nivel', 'estandarización_procedimiento_administrativo_nivel', 'estudiante_nivel', 'estudio_comparado_nivel', 'estudio_preinversión_nivel', 'etario_nivel', 'existente_nivel', 'expansión_emprendimiento_corporativo_nivel', 'experto_alto_nivel', 'financiamiento_nivel', 'financiero_nivel', 'fintech_nivel', 'formación_policial_nivel', 'funcionario_alto_nivel', 'gasto_nivel', 'gestión_turístico_nivel', 'grupo_etario_nivel', 'género_nivel', 'habilidad_digital_nivel', 'habilidad_nivel', 'hidrometrorológico_alto_nivel', 'hogar_bajo_nivel', 'hogar_nivel', 'hospital_segundo_nivel', 'hospital_tercero_nivel', 'identificación_experto_alto_nivel', 'impacto_financiero_nivel', 'implementación_sps_nivel', 'infantil_nivel', 'información_nivel', 'información_situacional_nivel', 'infraestructura_transporte_nivel', 'inmobiliario_nivel', 'institucional_nivel', 'institución_nivel', 'intervención_nivel', 'intradomiciliario_hogar_bajo_nivel', 'intrainstitucional_nivel', 'inventario_proyecto_nivel', 'inversión_público_nivel', 'inversión_seguridad_público_nivel', \
                    'investigación_alto_nivel', 'justicia_penal_nivel', 'laboratorio_nivel', 'mantenimiento_nivel', 'mayor_nivel', 'mecanismo_coordinación_nivel', 'medición_nivel', 'modalidad_app_nivel', 'modelo_vui_nivel', 'monitoreo_nivel', 'mujer_nivel', 'multiactor_nivel', 'ndc_nivel', 'nivel_aceptable_nivel', 'operacionalización_ndc_nivel', 'operación_nivel', 'organismo_nivel', 'paso_nivel', 'penal_nivel', 'pgn_nivel', 'pgn_organismo_nivel', 'plan_vigente_nivel', 'pluvial_nivel', 'pobreza_nivel', 'policial_nivel', 'porcentaje_hogar_nivel', 'preinversión_nivel', 'presencia_pgn_nivel', 'presupuestaria_nivel', 'principal_cambio_nivel', 'prioritario_nivel', 'problema_seguridad_alimentario_nivel', 'procedimiento_administrativo_nivel', 'proceso_relacionamiento_multiactor_nivel', 'promoción_nivel', 'proyección_aumento_nivel', 'proyecto_nivel', 'proyecto_prioritario_nivel', 'práctica_nivel', 'puesto_nivel', 'público_nivel', 'recurso_nivel', 'red_salud_nivel', 'reducción_nivel', 'referencia_emisión_nivel', 'reflejado_bajo_nivel', 'regulación_existente_nivel', 'relacionamiento_multiactor_nivel', 'relevante_nivel', 'representatividad_nivel', 'residual_nivel', 'resolutividad_primero_nivel', 'respuesta_nivel', 'riesgo_nivel', 'salud_nivel', 'salud_primero_nivel', 'salvadoreño_nivel', 'sector_aéreo_nivel', 'sector_fintech_nivel', 'seguridad_alimentario_nivel', 'seguridad_público_nivel', 'servicio_ciudadano_nivel', 'servicio_hidrometrorológico_alto_nivel', 'servicio_salud_primero_nivel', 'simplificación_trámite_nivel', 'sistema_financiamiento_nivel', 'sistema_nivel', 'situacional_nivel', 'social_nivel', 'sostenible_nivel', 'sps_nivel', 'tema_género_nivel', 'tercero_nivel', 'terminal_nivel', 'toma_decisión_nivel', 'transmisión_nivel', 'transporte_existente_nivel', 'transporte_nivel', 'trámite_nivel', 'turístico_nivel', 'unidad_salud_primero_nivel', 'vigente_nivel', 'violencia_nivel', 'índice_competitividad_nivel',  'aceptable_banco', 'acompañamiento_banco', 'administrado_banco', 'agenda_analítico_banco', 'analítico_banco', 'apoyo_banco', 'apoyo_financiero_banco', 'app_banco', 'aprendido_banco', 'aprobado_banco', 'banco_a', 'banco_alimento', 'banco_análisis', 'banco_argentino', 'banco_brasil', 'banco_carta', 'banco_carta_solicitud', 'banco_central', 'banco_central_ecuador', 'banco_centroamericano', 'banco_centroamericano_integración', 'banco_centroamericano_integración_económico', 'banco_ciudad', 'banco_ciudad_interesado', 'banco_colombia', 'banco_comercial', 'banco_comercio', 'banco_comercio_exterior', 'banco_compensación', 'banco_compensación_capacitivo', 'banco_componente', 'banco_comunal', 'banco_conocimiento', 'banco_cooperativo', 'banco_corresponsal', 'banco_dato', 'banco_desarrollo', 'banco_desarrollo_américa', 'banco_desarrollo_américa_latino', 'banco_desarrollo_caribe', 'banco_desarrollo_hondura', 'banco_digital', 'banco_digital_lámina', 'banco_digital_lámina_contexto', 'banco_distinto', 'banco_droga', 'banco_europeo', 'banco_europeo_inversión', 'banco_fase', 'banco_fase_preparación', 'banco_fin', 'banco_fomento', 'banco_interamericano', 'banco_interamericano_desarrollo', 'banco_inversión', 'banco_multilateral', 'banco_multilateral_desarrollo', 'banco_mundial', 'banco_nacional', 'banco_nacional_desarrollo', 'banco_nicaragua', 'banco_pago', 'banco_paraguay', 'banco_participante', 'banco_país', 'banco_primero_piso', 'banco_privado', 'banco_problema', 'banco_proyecto', 'banco_público', 'banco_público_desarrollo', 'banco_región', 'banco_república', 'banco_república_oriental', 'banco_resiliencia', 'banco_sangre', 'banco_sector', 'banco_segundo_piso', 'banco_servicio', 'banco_servicio_público', 'banco_subsector', 'banco_subsector_eléctrico', 'banco_unificado', 'banco_unificado_dato', 'capacidad_respuesta_operativo_banco', 'cartera_banco', 'comunicación_banco', 'conjunto_banco', 'conocimiento_banco', 'coordinación_banco', 'dominio_web_banco', 'economista_banco', 'educación_banco', 'equipo_banco', 'equipo_educación_banco', 'escenario_fiscal_recapitalización_banco', 'especialista_banco', 'estrategia_banco', 'estrategia_institucional_banco', 'estrategia_país_banco', 'estratégico_implementación_banco', 'expectativa_mercado_banco', 'experiencia_banco', 'financiado_banco', 'financiamiento_banco', 'financiero_banco', 'fiscal_recapitalización_banco', 'grupo_banco', 'hondura_apoyo_banco', 'hondura_banco', 'implementación_banco', 'infraestructura_social_financiado_banco', 'iniciativa_banco', 'institucional_banco', 'intelectual_banco', 'lección_aprendido_banco', 'mercado_banco', 'miembro_banco', 'miembro_prestatario_banco', 'migración_banco', 'millón_banco', 'no-objeción_previo_banco', 'objeción_banco', 'obra_financiado_banco', 'oficina_banco', 'operación_ays_banco', 'operación_banco', 'operativo_banco', 'participación_banco', 'país_banco', 'país_miembro_banco', 'país_miembro_prestatario_banco', 'país_prestatario_banco', 'personal_banco', 'política_banco', 'política_socioambiental_banco', 'portafolio_banco', 'posicionamiento_banco', 'preparación_banco', 'prestatario_banco', 'previo_banco', 'programa_banco', 'programa_operativo_banco', 'programa_srep_banco', 'propiedad_banco', 'propiedad_intelectual_banco', 'proyecto_banco', 'proyecto_financiado_banco', 'préstamo_banco', 'préstamo_recibido_banco', 'página_web_banco', 'recapitalización_banco', 'recibido_banco', 'recurso_banco', 'regionalización_banco', 'relación_banco', 'respuesta_operativo_banco', 'revisión_cartera_banco', 'rol_banco', 'satisfacción_banco', 'sector_banco', 'social_financiado_banco', 'socioambiental_banco', 'srep_banco', 'superintendencia_banco', 'suscrito_banco', 'viaje_personal_banco', 'web_banco', 'área_banco', 'aas', 'vaast', 'abaco', 'abandonado', 'ñeembucú', 'argumentación', 'útil', 'abanico', 'ñan', 'mañana', 'sobrepasado', 'apoyo_futuro', 'apoyo_futuro_empresario', 'futuro', 'futuro_empresario', 'futuro_operación', 'futuro_proyecto', 'habilidad_futuro', 'intervención_futuro', 'necesidad_futuro', 'programa_apoyo_futuro', 'programa_apoyo_futuro_empresario', 'abaratamiento', 'ítem', 'ítems', 'acompañado', 'bañado', 'bañado_sur', 'dañado', 'desempeñado', 'diseñado', 'diseñador', 'rediseñado', 'reseñado', 'acompañamiento_familiar_sdis_rediseñado', 'bañado_norte', 'bañado_sur', 'bañado_sur_asunción', 'bañado_tacumbú', 'digital_diseñado', 'diseñado_lineamiento', 'diseñado_lineamiento_técnico', 'diseñado_lineamiento_técnico_ejecutado', 'estrategia_gobierno_digital_diseñado', 'evento_diseñado', 'evento_diseñado_lineamiento', 'evento_diseñado_lineamiento_técnico', 'evento_diseñado_lineamiento_técnico_ejecutado', 'familiar_sdis_rediseñado', 'gobierno_digital_diseñado', 'implementación_mf_rediseñado', 'metodología_diseñado', 'mf_rediseñado', 'mf_rediseñado_a', 'modelo_diseñado', 'plan_diseñado', 'procedimiento_diseñado', 'rediseñado_a', 'rediseñado_servicio', 'rediseñado_servicio_integral', 'rediseñado_servicio_integral_desarrollo', 'resultado_modelo_diseñado', 'vivienda_bañado', 'vivienda_bañado_sur', 'vivienda_bañado_sur_asunción', 'ítalo', 'ñado', 'ón', 'íntimo_pareja', 'ís', 'ís_ico', 'óptimo', 'abaratamiento', 'abarcado', 'achiote', 'aci', 'acierto', 'acl', 'acnur', 'acogida', 'acogido', 'acogimiento', 'acometido', 'acompañado', 'acompañamiento_arranque', 'acompañamiento_banco', 'acompañamiento_bid', 'acompañamiento_cercano', 'acompañamiento_continuo', 'acompañamiento_estudio', 'acompañamiento_estudio_preinversión', 'acompañamiento_familia', 'acompañamiento_familia_programa', 'acompañamiento_familiar', 'acompañamiento_familiar_propuesto', 'acompañamiento_familiar_sdis', 'acompañamiento_familiar_sdis_rediseñado', 'acompañamiento_gestión', 'acompañamiento_gestión_riesgo', 'acompañamiento_implementación', 'acompañamiento_incentivo', 'acompañamiento_incentivo_fiscal', 'acompañamiento_mejora', 'acompañamiento_oferta', 'acompañamiento_oferta_servicio', 'acompañamiento_paciente', 'acompañamiento_país', 'acompañamiento_perfil', 'acompañamiento_perfil_validado', 'acompañamiento_planificación', 'acompañamiento_planificación_estratégico', 'acompañamiento_primero_ronda', 'acompañamiento_proceso', 'acompañamiento_proceso_socialización', 'acompañamiento_psicológico', 'acompañamiento_situ', 'acompañamiento_social', 'acompañamiento_socio-organizacional', 'acompañamiento_técnico', 'ínfimo', 'ínsita', 'íntegro', 'íntimo', 'índole', 'abasto', 'abc', 'abebo', 'abiec', 'ability', 'ability_to', 'abm', 'abo', 'aboal', 'abocado', 'abonado', 'abordado', 'abordaje', 'abordaje_fiscal_post', 'abordaje_fiscal_post_pandemia', 'about', 'abr', 'abramo', 'abreu', 'abreviatura', 'abreviaturo', 'abril', 'abrupto', 'absceso', 'absoluto', 'absorción', 'absorción_tecnológico', 'abstención', 'abstracción', 'abstracto', 'abundancia', 'abundante', 'abuso', 'ac', 'academia', 'academy', 'académico', 'académico_oportunidad', 'acajutla', 'acaparamiento', 'acaray', 'acarreo', 'acb', 'acc', 'acce', 'acceder', 'accelerator', 'accepted', 'accesibilidad_servicio', 'accesibilidad_universal', 'accesible', 'acceso_acceso', 'acceso_acceso_activo', 'acceso_activo', 'acceso_activo_necesario',  'acceso_capital', 'which', 'what', 'who', 'whose', 'customer_is_an', \
                                       'service_customer_is_an', 'where', 'acción_adaptación', 'acción_b', 'acción_colectivo', 'acción_concreto', 'acción_control', 'acción_correctivo', 'acción_correctivo_riesgo', 'acción_correctivo_riesgo_encontrado', 'acción_ee', 'acción_especifico', 'acción_estratégico', 'acción_fiscalización', 'acción_fortalecimiento', 'acción_género', 'acción_humano', 'acción_ilícito', 'acción_implementación', 'acción_ipg', 'acción_manera_directo', 'acción_mejora', 'acción_mejora_continuo', 'acción_mitigación', 'acción_política', 'acción_prevención', 'acción_prioritario', 'acción_seguridad', 'acción_seguridad_cibernético', 'acción_tratamiento', 'acción_tratamiento_riesgo', 'acción_vinculado', 'afirmado_interacción', 'alto_costo_transacción', 'atracción', 'atracción_ied', 'atracción_inversión', 'calefacción', 'coacción', 'contracción', 'contracción_demanda', 'contracción_económico', 'costo_transacción', 'diseño_plan_acción', 'eje_acción', 'extracción', 'factor_contracción', 'fracción', 'fracción_segundo_decisión', 'implementación_acción', 'implementación_plan_acción', 'importante_acción', 'importante_acción_tratamiento', 'infracción', 'insatisfacción', 'interacción', 'interacción_usuario', 'ley_transacción', 'ley_transacción_electrónico', 'línea_acción', 'plan_acción', 'plan_acción_correctivo', 'plan_acción_género', 'plan_acción_implementación', 'plan_acción_ipg', 'reacción', 'redacción', 'reducción_costo_transacción', 'registro_acción', 'retracción', 'satisfacción', 'satisfacción_ciudadano', 'unidad_acción', 'adquisición_estándar_eldesarrollo_software', 'apo10.05_isa', 'apo10.05_iso', 'apo12.02_iso', 'apo12.06_isa', 'apo12.06_iso', 'dss01.05_isa', 'dss02.02_isa', 'dss03.01_isa', 'dss03.04_isa', 'dss03.04_iso', 'dss04.02_isa', 'dss04.04_isa', 'dss05.01_isa', 'dss05.02_isa', 'dss05.04_isa', 'dss05.06_isa', 'dss05.07_isa', 'dss06.06_isa', 'gi.5_incidente', 'gi.5_incidente_seguridad', 'ir-8_gi.5', 'ir-8_gi.5_incidente', 'p.i.b._actual', 'pm-11_gr.2', 'pm-13_gh.2', 'pm-16_gr.2', 'si-4_so.7', 'so.5_software', 'so.5_software_malicioso', 'ccn-cert_ia_09-15', 'ccn-cert_informe', 'adquisición_estándar_eldesarrollo', 'eldesarrollo', 'eldesarrollo_software', 'eldesarrollo_software_resiliencia', 'internet_cooperación_formal', 'internet_cooperación_formal_cooperación', 'internet_cooperación_formal_cooperación_informal', 'internet_cooperación_formal_cooperación_informal_marco', 'tenencio', 'angelelli', 'cibernético_legislación_depropiedad_intelectual_legislación_procesalcontro', 'intelectual_legislación_procesalcontro', 'intelectual_legislación_procesalcontro_delito', 'intelectual_legislación_procesalcontro_delito_cibernético', 'legislación_depropiedad_intelectual_legislación_procesalcontro', 'legislación_procesalcontro', 'legislación_procesalcontro_delito', 'legislación_procesalcontro_delito_cibernético', 'legislación_procesalcontro_delito_cibernético_privacidad', 'procesalcontro', 'consumidor_legislación_sustantivacontra', 'consumidor_legislación_sustantivacontra_delito', 'consumidor_legislación_sustantivacontra_delito_cibernético', 'legislación_protecciónal_consumidor_legislación_sustantivacontra', 'legislación_sustantivacontra', 'legislación_sustantivacontra_delito', 'legislación_sustantivacontra_delito_cibernético', 'legislación_sustantivacontra_delito_cibernético_legislación', 'protecciónal_consumidor_legislación_sustantivacontra', 'protecciónal_consumidor_legislación_sustantivacontra_delito', 'sustantivacontra_delito', 'sustantivacontra_delito_cibernético', 'sustantivacontra_delito_cibernético_legislación', 'sustantivacontra_delito_cibernético_legislación_depropiedad', 'acelerado', 'acelerador', 'aceleradora', 'acentuado', 'accionable', 'accionamiento', 'accionar', 'accionario', 'costo_transaccional', 'fraccionamiento', 'público_transaccional', 'servicio_público_transaccional', 'servicio_transaccional', 'transaccional', 'aceptable', 'aceptable_activo', 'nivel_aceptable', 'aceptado', 'acrónimo', 'acrónimo_inglés', 'acrónimo_norma', 'act', 'acta',    'acting', 'action', 'actions', 'actitud', 'actitudinal', 'activación',    'activad', 'activador', 'activar', 'active', 'activedirectory',    'activi', 'necesario_actividad', 'necesario_actividad_operador', 'objetivo_actividad', 'parte_interesado_actividad', 'perfil_amenaza_actividad', 'priorización_actividad', 'priorización_actividad_seguridad', 'práctica_documentado_actividad', 'rama_actividad', 'realización_actividad', 'resultado_específico_actividad', 'resultado_específico_actividad_técnico', 'riesgo_actividad', 'riesgo_actividad_operador', 'sector_eléctrico_américa_latino_actividad', 'soporte_actividad', 'soporte_actividad_recuperación', 'subactividad', 'interesado_actividad', 'interesado_actividad_gestión', 'latino_actividad', 'lección_aprendido_actividad', 'lección_aprendido_actividad_futuro', 'mejora_actividad', 'mejora_actividad_respuesta', 'implementación_actividad', 'importante_actividad', 'importante_actividad_operador', 'inactividad', 'documentado_actividad', 'documentado_actividad_gestión', 'ejecución_actividad', 'ejecución_actividad_apropiado', 'eléctrico_américa_latino_actividad', 'específico_actividad', 'específico_actividad_técnico', 'activo_importante_actividad', 'activo_necesario_actividad', 'amenaza_actividad', 'amenaza_actividad_operador', 'américa_latino_actividad', 'aprendido_actividad', 'aprendido_actividad_detección', 'aprendido_actividad_futuro', 'categoría_resultado_específico_actividad', 'conjunto_actividad', 'continuidad_actividad', 'continuidad_actividad_operativo', 'criterio_riesgo_actividad', 'cumplimiento_actividad', 'desempeño_actividad', 'desempeño_actividad_acceso', 'desempeño_actividad_gestión', 'evaluación_actividad', 'evento_actividad', 'indicador_actividad', 'actividad_crucial',
 'actividad_crucial_protección', 'crucial', 'crucial_protección', 'crucial_protección_infraestructura']
len(set(final_stop_words))

In [ ]:
terms_to_remove = terms_to_remove + final_stop_words
#'componente_iia', 'asistencia_técnico', 'capacidad_institucional', 'estrategia_institucional', 'área_transversal', 'estructurar_ejecución', \
# 'componente_descripción', 'operación_préstamo', 'organismo_ejecutor', 'américa_latina', 'cooperación_técnico', 'cooperación_técnica', 'operación_de_cooperación', 'agencia_ejecutora', \
#  'operación_de_cooperación_técnico', 'agencia_ejecutor', 'nivel_mundial', 'organismo_ejecutor', 'unidad_ejecutor', 'resultar_esperar', 'producto_esperar', 'esperar_del_componente', \
#                                     'resultados_esperar', 'resultar_esperar_del_componente', 'principal_resultar_esperar', 'poner_en_funcionamiento', 'problema_específico', \
#                                     'nivel_nacional', 'autoridad_nacional', 'presentar_operación', 'resultar_anual', 'estructurar_organizacional', 'gobernar_central', 'tomar_de_decisión'\
#                                    'adicional_con_programar', 'documento_de_marco_sectorial', 'adquisición_de_insumo', 'proyectar_pilotar', 'alto_impactar', 'efectividad_comparar', \
#                                    'modelar_de_negociar', 'término_de_referenciar', 'aumentar_sostener', 'aumentar_inicial', 'aumentar_del_nivel', 'lección_aprender', 'partir_interesar', \
#                                    'resultar_desear', 'centralizar_capaz', 'área_relacionar', 'información_utilizar', 'decisión_relacionar', 'generación_transmisiónuna_combinación', \
#                                    'meta_b']
terms_to_remove = list(set(terms_to_remove))
len(terms_to_remove)

In [ ]:
df_base

#### 4.1.2: Remove selected terms

In [ ]:
df_base['alt2_terms'] = ''

In [ ]:
%%time
for index, row in df_base.iterrows():
    #print('Processing index:', str(index))
    df_base.at[index, 'alt2_terms'] = [word for word in df_base['alt2_list_terms_expanded'][index] if word not in terms_to_remove and '_' in word]
    #print([word for word in df_base['list_of_terms'][index] if word not in terms_to_remove])

#### 4.1.3: Check results

In [ ]:
terms_final = df_base.alt2_terms.to_list()
terms_final_flat = []
for i in range(len(terms_final)):
    for token in terms_final[i]:
        terms_final_flat.append(token)
len(set(terms_final_flat))

In [ ]:
terms_final_flat = Counter(terms_final_flat)
sort_orders_terms_final = sorted(terms_final_flat.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_terms_final:
    print(i[0], i[1])

<br>
<br>

### 4.2: Tokens

In [ ]:
pprint(data_lemmatized[0])

In [ ]:
#### 4.2.1: Check results

In [ ]:
# Word evaluation:
word_stats_only_tokens = []
for i in range(len(data_lemmatized)):
    for token in data_lemmatized[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats_only_tokens.append(token)

In [ ]:
len(set(word_stats_only_tokens))

In [ ]:
# custom stop_words:
final_stop_words.extend(['ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'ill', 'descripción', 'componente', 'objetivo', 'ct', 'mailto', 'país', 'millón', 'millones', \
                   'año', 'años', 'dólar', 'dolar', 'dólares', 'si', 'bid', 'us', 'oc', 'gn', 'tc', 'atn', 'opc', 'pib', 'ar', 'br', 'uy', 'cl', 'co', \
                   'cclip', 'pbl', 'uis', 'ab', 'org', 'pr', 'bo', 'bl', 'pe', 'ec', 'ja', 'mx', 'ca', 'gu', 'su', 'ho', 'hn', 'mr', 'rg', 'ee', 'uu', \
                   'cr', 'tdr', 'rn', 'nº', 'usd', 'gy', 've', 'et', 'the', 'for', 'to', 'grt', 'fm', 'pr', 'pa', 'ni', 'aa', 'es', 'sp', 'us', 'través'\
                   'inglés', 'cty', 'nv', 'profisco', 'asimismo', 'actual', 'costo', 'resultado', 'esperado', 'ejecutar', 'ejecutado', 'unidad', 'agencia', 'justificación', \
                   'véase', 'tc', 'dela', 'enel', 'sobrar', 'of', '▪', '.', '+', '¶','$','','=','€', 'desarrollo', 'servicio', 'país', 'proyecto', 'técnico', 'ejemplo',\
                    'd', 'xx', 'ib', 'nº', 'gda', 'kr', 'wsa', 'jsf', 'such'])

final_stop_words = final_stop_words + ['iec', 'iso', 'nist', 'pr', 'sr', 'apo', 'isa', 'sp', 'cobit', 'rev', 'the', 'dss', 'and', 'of', 'csc', 'cis', 'or', 'vez', 'itu', 'usuarioen', 'sc', 'bai', 'ca', 'ref', 'año', 'co', 'sin', 'embargo', 'an', 'ac', 'respecto', 'pm', 'cabo', 'banco', 'sa', 'inglés', 'cp', 'ic', 're', 'subsector', 'subcategoría', 'rsi', 'tanto', 'ia', 'sigla', 'eldesarrollo', 'sd', 'cm', 'pi', 'pe', 'rc', 'is', 'ens', 'mc', 'gi', 'for', 'cuanto', 'capabilitie', 'so', 'día', 'by', 'that', 'ccn', 'ra', 'gr', 'ver', 'type', 'ga', 'csi', 'oea', 'es', 'ps', 'caribe', 'bid', 'are', 'au', 'ae', 'on', 'note', 'as', 'mes', 'cf', 'be', 'tal', 'decir', 'sf', 'one', 'acrónimo', 'pd', 'taller', 'españa', 'which', 'ad', 'cs', 'ma', 'agencia', 'etapa', 'bahama', 'go', 'mea', 'er', 'tp', 'ión', 'ción', 'solo', 'pl', 'mp', 'diciembre', 'guyana', 'costa', 'chile', 'último', 'panamá', 'with', 'physical', 'can', 'gh', 'falta', 'pesar', 'europa', 'not', 'octubre', 'junio', 'préstamo', 'barbado', 'brasil', 'república', 'acreditación', 'from', 'category', 'plazo', 'hoc', 'program', 'abril', 'enero', 'san', 'edm', 'bia', 'enisa', 'semana', 'ddo', 'inc', 'mano', 'pregunta', 'eps', 'gracias', 'trinidad', 'simposio', 'salvador', 'espíritu', 'santo', 'another', 'more', 'at', 'reunión', 'conferencia', 'fecha', 'idente', 'obstante', 'idad', 'marzo', 'scrm', 'dado', 'ministro', 'based', 'budapest', 'saint', 'nevis', 'provided', 'using', 'this', 'such', 'herrmann', 'uruguay', 'site', 'segur', 'it', 'ion', 'aa', 'noviembre', 'argentino', 'belice', 'haití', 'paraguay', 'rd', 'kitt', 'granadina', 'pgp', 'used', 'any', 'where', 'least', 'related', 'institute', 'uy', 'rp', 'iberseguridad', 'corea', 'ico', 'nes', 'también', 'torno', 'lidad', 'def', 'coord', 'ibersegur', 'lar', 'ds', 'mancomunidad', 'mayo', 'barbuda', 'ii', 'septiembre', 'julio', 'ja', 'perú', 'venezuela', 'par', 'pro', 'crl', 'day', 'owner', 'cnss', 'include', 'applications', 'object', 'other', 'may', 'but', 'their', 'have', 'between', 'order', 'need', 'mua', 'ieee', 'cip', 'latinoamérica', 'ge', 'safe', 'reino', 'ue', 'cr', 'recuperac', 'iber', 'bg', 'febrero', 'policy', 'belize', 'colombia', 'iii', 'granada', 'gy', 'mx', 'santo', 'cesión', 'em', 'presa', 'mente', 'coa', 'dad', 'nado', 'assigned', 'responsibility', 'official', 'inst', 'ability', 'example', 'defined', 'some', 'set', 'party', 'relationship', 'does', 'might', 'support', 'operation', 'customers', 'drae', 'needed', 'run', 'capabilities', 'performance', 'target', 'provide', 'also', 'measure', 'organization', 'cnssi_', 'privilege', 'its', 'know', 'modification', 'default', 'most', 'eco', 'subdivisión', 'csf', 'american', 'ansi', 'abm', 'abreviatura', 'setiembre', 'art', 'north', 'corporation', 'good', 'appropriate', 'measur', 'rs', 'man', 'cci', 'ucrania', 'trend', 'root', 'corp', 'sur', 'situ', 'acls', 'tación', 'tructura', \
                                      'ís', 'responsabi', 'icac', 'ica', 'cana', 'iva', 'lataforma', 'igac', 'fac', 'iesgo', 'serv', 'ios', 'desarro', 'tecno', 'us', 'kiviat', 'fa', 'cea', 'ee', 'cps', 'usc', 'cmm', 'loan', 'pbl', 'ba', 'israel', 'indio', 'ran', 'caminoa', 'mintic', 'india', 'ecuador', 'nº', 'gob', 'guatemala', 'get', 'guyanés', 'francés', 'conatel', 'hondura', 'mexico', 'mitic', 'agosto', 'dpsm', 'suriname', 'tt', 'cita', 'ne', 'lopd', 'tre', 'trato', 'favor', 'español', 'res', 'pue', 'cabida', 'sis', 'mas', 'ans', 'coe', 'otan', 'iv', 'necessarily', 'under', 'put', 'behalf', 'out', 'actions', 'either', 'both', 'overall', 'them', 'dsaas', 'when', 'certain', 'conditions', 'tests', 'instance', 'mentira', 'generally', 'applied', 'sec', 'against', 'objetivo', 'objective', 'levels', 'ss', 'antel', 'ceibal', 'ínsita', 'emg', 'agesic', 'firewa', 'll', 'bidbanco', 'oeaorganización', 'guidela', 'gridsla', 'iisf', 'consortium', 'iic', 'iiot', 'crf', 'unece', 'united', 'nations', 'economic', 'commission', 'deming', 'act', 'erm', 'purduemodelo', 'purdue', 'dnp', 'opc', 'ua', 'gridred', 'dcssistemas', 'rtuunidad', 'ataqueconjunto', 'ataquemétodo', 'idssistema', 'ais', 'central', 'siemsistema', 'soccentro', 'cisodirector', 'middle', 'alc', 'industroyer', 'turquía', 'farewell', 'bellingham', 'australia', 'david', 'salt', 'sobig', 'csx', 'sasser', 'british', 'airway', 'delta', 'tehama', 'colusa', 'dugu', 'gauss', 'dragon', 'exxon', 'shell', 'bp', 'hemisferio', 'oriente', 'asia', 'zotob', 'daimler', 'chrisler', 'ag', 'kwc', 'onion', 'city', 'probalidad', 'rtic', 'ipa', 'nte', 'respuestatoda', 'instar', 'lación', 'proba', 'segmen', 'infra', 'infraestructu', 'op', 'cor', 'pico', 'xx', 'hart', 'eng', 'regis', 'tra', 'end', 'point', 'respuestano', 'cisne', 'cercanía', 'transmisiónuna', 'despositivo', 'adición', 'tan', 'pt', 'edifico', 'restrición', 'un sitio', 'aplicacio', 'billón', 'usd', 'proteccióna', 'naciona', 'igar', 'lega', 'lmente', 'lu', 'jo', 'lat', 'hac', 'abr', 'preparac', 'automát', 'env', 'laborat', 'irtua', 'invest', 'izac', 'mit', 'contro', 'ive', 'orquestac', 'recuperaciónc', 'inac', 'especia', 'ías', 'inado', 'mater', 'imulac', 'inada', 'detecc', 'acc', 'restaurac', 'compet', 'habi', 'desarrol', 'istema', 'prote', 'jan', 'conf', 'idenc', 'ing', 'exce', 'lenc', 'profe', 'doe', 'dhs', 'mil', 'ef', 'ej', 'sumin', 'iclo', 'ponibilidad', 'vulnerabilida', 'dcpd', 'hr', 'dólar', 'rmp', 'fu', 'nc', 'ar', 'ross', 'et', 'quehacer', 'oxford', 'organizacionesy', 'cumbre', 'siglo', 'xxi', 'chino', 'icic', 'pwc', 's', 'puc', 'bolivia', 'cgii', 'br', 'numeral', 'ente', 'colcert', 'colombiano', 'delegatura', 'europol', 'micitt', 'desde', 'costarricense', 'dominiqués', 'dominica', 'arcotel', 'ecucert', 'deloitte', 'sv', 'encargada', 'grenada', 'luz', 'gt', 'getsafeonline', 'summer', 'incibe', 'jamaica', 'mstem', 'sarrollo', 'mexicano', 'citi', 'py', 'pcm', 'peruano', 'kittsand', 'dpm', 'ttconnect', 'gub', 'suscerte', 'sede', 'sucer', 'venezolano', 'cos', 'ceptible', 'uti', 'dida', 'grama', 'có', 'digo', 'infor', 'mación', 'cas', 'ejem', 'plo', 'jun', 'profesiona', 'poste', 'cer', 'na', 'moneda', 'lssi', 'edi', 'electronic', 'actuali', 'congéner', 'riante', 'comunicacio', 'ghz', 'trozo', 'tico', 'gan', 'cho', 'abo', 'gado', 'di', 'nero', 'kilómetro', 'sue', 'longi', 'interesar', 'conocimien', 'ci', 'frado', 'intimi', 'activi', 'traseña', 'car', 'pio', 'embar', 'ciu', 'mos', 'progra', 'cues', 'tión', 'db', 'utili', 'minio', 'ñan', 'd', 'perso', 'transferen', 'cia', 'gar', 'glés', 'miento', 'apli', 'infrastructu', 'grafía', 'transaccio', 'cla', 'za', 'sqldefinición', 'vali', 'dación', 'intru', 'tec', 'nología', 'cortafue', 'landefinición', 'elenco', 'admi', 'nistración', 'emisor', 'seguri', 'indi', 'tarje', 'ssc', 'nera', 'dose', 'apa', 'renta', 'cumento', 'dar', 'siste', 'gocio', 'tele', 'termediario', 'tario', 'mini', 'proce', 'dente', 'servi', 'cio', 'ri', 'vest', 'shamir', 'adleman', 'ase', 'mencio', 'usua', 'rio', 'ciberdelincuen', 'dress', 'fre', 'cuencia', 'ssldefinición', 'ingle', 'transport', 'fun', 'ciona', 'propagar', 'direc', 'soft', 'ware', 'men', 'ministrador', 'orde', 'tocol', 'tar', 'zero', 'ciberdelin', 'ter', 'sci', 'subsanación', 'configuracio', 'ied', 'atlántico', 'norte', 'igf', 'foc', 'ccd', 'ass', 'om', 'mcafee', 'sl', 'isdefe', 'loud', 'offered', 'invoked', 'possess', 'qualities', 'purpose', 'imply', 'reason', 'was', 'resulted', 'exercising', 'published', 'contains', 'operate', 'made', 'obtained', 'publicly', 'however', 'produced', 'likely', 'following', 'principles', 'unless', 'there', 'provisions', 'contrary', 'activitie', 'mak', 'includ', 'uses', 'entitie', 'deviz', 'csv', 'who', 'compaa', 'than', 'small', 'ón', 'own', 'sport', 'golf', 'easily', 'without', 'required', 'reenter', 'ease', 'moving', 'essence', 'here', 'achieved', 'exactly', 'format', 'accepted', 'even', 'if', 'formats', 'simple', 'straightforward', 'achieve', 'commonly', 'rekeying', 'could', 'described', 'easy', 'three', 'representations', 'manner', 'suitable', 'guide', 'clic', 'cabalgamiento', 'two', 'different', 'instrument', 'engind', 'compound', 'therefore', 'useful', 'whose', 'existence', 'strongly', 'various', 'together', 'iaa', 'underlying', 'over', 'deployed', 'components', 'host firewalls', 'eufemismo', 'ligthweight', 'numbers', 'symbols', 'entities', 'describir', 'according', 'clear', 'rules', 'repeated', 'measuring', 'comparison', 'reference', 'measurements', 'they', 'orient', 'decisions', 'better', 'understanding', 'casual', 'relationships', 'intended', 'expectations', 'observed', 'facts', 'number', 'symbol', 'characterize', 'attribute', 'compensating', 'technical', 'safeguard', 'countermeasure', 'employed', 'lieu', 'recommended', 'low', 'moderate', 'high', 'baselín', 'provid', 'equivalent', 'decimal', 'metric', 'proposed', 'facilitate', 'decisio', 'n', 'making', 'improve', 'through', 'reporting', 'relevant', 'ribagorda', 'principle', 'should', 'each', 'granted', 'minimum', 'needs', 'multi', 'computations', 'terms', 'express', 'same', 'while', 'processe', 'feature', 'automatically', 'sesgo', 'picaresca', 'paas', 'supported', 'single', 'ce', 'potentially', 'harbor', 'serve', 'statutory', 'staff', 'directs', 'today', 'responsible', 'among', 'organizational', 'elements', 'involved', 'derived', 'after', 'agreed', 'period', 'ver_pag', 'whether', 'int', 'many', 'lead', 'what', 'actually', 'result', 'the', 'precise', 'meaning', 'remain', 'identifies', 'established', 'omb', 'appendix', 'spp', 'saa', 'fip', 'troya', 'erc', 'ksk', 'priori', 'cc', 'lucia']
final_stop_words = list(set(final_stop_words))

In [ ]:
len(final_stop_words)

In [ ]:
## Check the tokens:
tokens_final_flat = []
for i in range(len(data_lemmatized)):
    for token in data_lemmatized[i]:
        tokens_final_flat.append(token)

In [ ]:
tokens_final_flat = Counter(tokens_final_flat)
sort_orders_tokens_final = sorted(tokens_final_flat.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_tokens_final:
    print(i[0], i[1])

In [ ]:
## filter out tokens
df_base['alt2_tokens'] = ''

In [ ]:
%%time
for index, row in df_base.iterrows():
    #print('Processing index:', str(index))
    df_base.at[index, 'alt2_tokens'] = [word for word in data_lemmatized[index] if word not in final_stop_words]
    #print([word for word in df_base['list_of_terms'][index] if word not in terms_to_remove])

In [ ]:
## tokens - final check

In [ ]:
# Convert to list
tokens_final = df_base['alt2_tokens'].values.tolist()

tokens_final_flat = []
for i in range(len(tokens_final)):
    for token in tokens_final[i]:
        tokens_final_flat.append(token)

len(set(tokens_final_flat))

In [ ]:
tokens_final_flat = Counter(tokens_final_flat)
sort_orders_tokens_final = sorted(tokens_final_flat.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_tokens_final:
    print(i[0], i[1])

# **************************************************************************************************************** #
<br>
<br>
<br>

### 5. Merge tokens and terms/n-Grams

In [ ]:
df_base.reset_index(drop=True, inplace=True)

In [ ]:
# previously obtained terms/n-grams are added to the dataset
data_lemmatized_full = []
for index, row in df_base.iterrows():
    data_lemmatized_full.append(df_base['alt2_tokens'][index] + df_base['alt2_terms'][index])

In [ ]:
len(data_lemmatized_full)

In [ ]:
# Word evaluation:
word_stats = []
for i in range(len(data_lemmatized_full)):
    for token in data_lemmatized_full[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats.append(token)

In [ ]:
word_stats = Counter(word_stats)
sort_orders = sorted(word_stats.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders:
    print(i[0], i[1])

# **************************************************************************************************************** #
<br>
<br>
<br>

In [ ]:
# Adding data_lemmatized as a new column and store the results:
df_base['alt2_data_lemmatized'] = data_lemmatized_full

In [ ]:
df_base

# **************************************************************************************************************** #
<br>
<br>
<br>

### Store results

In [ ]:
df_base.columns

In [ ]:
df_result = df_base[['doc_type', 'language', 'OPERATION_NUMBER', 'DOCUMENT_REFERENCE',
       'extracted', 'alt2_list_terms_base', 'alt2_list_terms',
       'alt2_tokens_pre_clean', 'alt2_list_terms_expanded', 'alt2_terms',
       'alt2_tokens', 'alt2_data_lemmatized']].copy()
df_result.head()

In [ ]:
# v1.2: Store df_base, in English, containing terms and tokens
f_df_result = 'nlp_df_result_tokens_terms_2021-01-22_english_v1.2.joblib'
joblib.dump(df_result, './output/' + f_df_result + '.bz2', compress=('bz2', 3))#

In [ ]:
import pickle

In [ ]:
## v1.2: Tokens (English)
with open('./output/nlp_operations_tokens_2021-01-22_english_v1.2.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized_full, filehandle)

In [ ]:
#### FIN ######

# **************************************************************************************************************** #
<br>
<br>
<br>

In [ ]:
######

In [ ]:
import gensim

In [ ]:
#load word2vec model
from gensim.models.keyedvectors import KeyedVectors

#from gensim.models.word2vec import Word2Vec

#model=word2vec.KeyedVectors.load_word2vec_format('../Kiwicha/Word2Vec_Spanish_Model/keyed_vectors/complete.kv.vectors.npy', \
#                                                 binary=True)

#model = Word2Vec.load("../Kiwicha/Word2Vec_Spanish_Model/complete.model")
model = gensim.models.KeyedVectors.load_word2vec_format('../Kiwicha/NLPL source/model.txt', binary=False, unicode_errors='replace')

In [ ]:
model.most_similar('ciberseguridad', topn=10)

In [ ]:
from sklearn.metrics import jaccard_score

#phrases=["seguridad informática","seguridad cibernética","ciberseguridad","seguridad de la información","sistema informático","ransomware"]
#query="ataque cibernético"

#phrases=["seguridad informática","seguridad cibernética","ciberseguridad", "cybersecurity"]
#query="seguridad de la información"

phrases = ["gases de efecto invernadero", "calentamiento global", "capa de ozono"]
query = "emisiones de carbono"

results_word2vec=[]
results_jaccard=[]

def compute_jaccard(t1,t2):
    
    intersect = [value for value in t1 if value in t2] 
    
    union=[]
    union.extend(t1)
    union.extend(t2)
    union=list(set(union))
    
    
    jaccard=(len(intersect))/(len(union)+0.01)
    return jaccard
    

for p in phrases:
    tokens_1=[t for t in p.split() if t in model.vocab]
    tokens_2=[t for t in query.split() if t in model.vocab]
    
    #compute jaccard similarity
    jaccard=compute_jaccard(tokens_1,tokens_2)
    results_jaccard.append([p,jaccard])
    
    #compute cosine similarity using word embedings 
    cosine=0
    if (len(tokens_1) > 0 and len(tokens_2)>0):
        cosine=model.n_similarity(tokens_1,tokens_2)
        results_word2vec.append([p,cosine])

print("Phrases most similar to '{0}' using word2vec word embeddings".format(query))
pd.DataFrame(results_word2vec,columns=["phrase","score"]).sort_values(by=["score"],ascending=False)

In [ ]:
results_jaccard

In [ ]:
model.most_similar(['carbono'], topn=10)

In [ ]:
model.most_similar('carbono', topn=10)

In [ ]:
'gases' in model.vocab

In [ ]:
model.most_similar(['gases'], topn=10)

In [ ]:
del model
# end alternative tests (11/01)

In [ ]:
#######

In [ ]:
for i in sort_orders_terms:
    if 'utilizado' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'definido' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'sigla' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'útil' in i[0]:
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if i[0].startswith('único_'):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if (i[0].startswith('._') or i[0].endswith('_.') or '.' in i[0]):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if (i[0].startswith('-_') or i[0].endswith('_-') or i[0].startswith('−_')):
        print(i[0], i[1])
        terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if (i[0].startswith('órgano_')):
        print(i[0], i[1])
        #terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    #if (i[0].startswith('-_') or i[0].endswith('_-') or i[0].startswith('−_')):
    if 'organización' in i[0]:
        print(i[0])#, i[1])
        #terms_to_remove.append(i[0])

In [ ]:
'asistencia_técnico', 'capacidad_institucional', 'estrategia_institucional', 'área_transversal', 'estructurar_ejecución', \
 'componente_descripción', 'operación_préstamo', 'organismo_ejecutor', 'américa_latina', 'cooperación_técnico', 'cooperación_técnica', 'operación_de_cooperación', 'agencia_ejecutora', \
  'operación_de_cooperación_técnico', 'agencia_ejecutor', 'nivel_mundial', 'organismo_ejecutor', 'unidad_ejecutor', 'resultar_esperar', 'producto_esperar', 'esperar_del_componente', \
                                    'resultados_esperar', 'resultar_esperar_del_componente', 'principal_resultar_esperar', 'poner_en_funcionamiento', 'problema_específico', \
                                    'nivel_nacional', 'autoridad_nacional', 'presentar_operación', 'resultar_anual', 'estructurar_organizacional', 'gobernar_central', 'tomar_de_decisión',\
                                    'adicional_con_programar', 'documento_de_marco_sectorial', 'adquisición_de_insumo', 'proyectar_pilotar', 'alto_impactar', 'efectividad_comparar', \
                                    'modelar_de_negociar', 'término_de_referenciar', 'aumentar_sostener', 'aumentar_inicial', 'aumentar_del_nivel', 'lección_aprender', 'et_al', 'new_area', \
                                    'technical_assistance', 'optional_link', 'good_practice', 'sector_framework', 'action_plan', 'short_term', 'long_term', 'medium_term', 'year_implementation'\
                                    'tc_resource', 'year_action', 'grant_operation', 'year_of_age'

In [ ]:
test = []
for i in sort_orders_terms:
    #if  i[0].startswith('banco_') or i[0].endswith('_banco'): # (i[0].startswith('nivel_') or or i[0].startswith('−_')):
    if 'acompa' in i[0]:
       # print(i[0], i[1])
        test.append((i[0], i[1]))
sorted(test)

In [ ]:
sorted([item[0] for item in test])

In [ ]:
test = []
for i in sort_orders_terms:
    #if (i[0].startswith('-_') or i[0].endswith('_-') or i[0].startswith('−_')):
    if 'préstamo' in i[0]:
       # print(i[0], i[1])
        test.append((i[0], i[1]))
sorted(test)

In [ ]:
for i in sort_orders_terms:
    #if (i[0].startswith('-_') or i[0].endswith('_-') or i[0].startswith('−_')):
    if '+' in i[0]:
        print(i[0], i[1])
        #terms_to_remove.append(i[0])

In [ ]:
for i in sort_orders_terms:
    #if (i[0].startswith('-_') or i[0].endswith('_-') or i[0].startswith('−_')):
    if 'aplicación' in i[0]:
        print(i[0], i[1])
        #terms_to_remove.append(i[0])

In [ ]:
#terms_to_remove = terms_to_remove + ['componente_iia', 'asistencia_técnico', 'capacidad_institucional', 'estrategia_institucional', 'área_transversal', 'estructurar_ejecución', \
# 'componente_descripción', 'operación_préstamo', 'organismo_ejecutor', 'américa_latina', 'cooperación_técnico', 'cooperación_técnica', 'operación_de_cooperación', 'agencia_ejecutora', \
#  'operación_de_cooperación_técnico', 'agencia_ejecutor', 'nivel_mundial', 'organismo_ejecutor', 'unidad_ejecutor', 'resultar_esperar', 'producto_esperar', 'esperar_del_componente', \
#                                     'resultados_esperar', 'resultar_esperar_del_componente', 'principal_resultar_esperar', 'poner_en_funcionamiento', 'problema_específico', \
#                                     'nivel_nacional', 'autoridad_nacional', 'presentar_operación', 'resultar_anual', 'estructurar_organizacional', 'gobernar_central', 'tomar_de_decisión'\
#                                    'adicional_con_programar', 'documento_de_marco_sectorial', 'adquisición_de_insumo', 'proyectar_pilotar', 'alto_impactar', 'efectividad_comparar', \
#                                    'modelar_de_negociar', 'término_de_referenciar', 'aumentar_sostener', 'aumentar_inicial', 'aumentar_del_nivel', 'lección_aprender', 'partir_interesar', \
#                                    'resultar_desear', 'centralizar_capaz', 'área_relacionar', 'información_utilizar', 'decisión_relacionar', 'generación_transmisiónuna_combinación', \
#                                    'meta_b']

In [ ]:
terms_to_remove = list(set(terms_to_remove))

In [ ]:
len(terms_to_remove)

#### 2.5. Remove selected terms

In [ ]:
df_base['alt2_terms'] = ''

In [ ]:
%%time
for index, row in df_base.iterrows():
    #print('Processing index:', str(index))
    df_base.at[index, 'alt2_terms'] = [word for word in df_base['alt2_list_terms'][index] if word not in terms_to_remove and '_' in word]

#### 2.6. Check results

In [ ]:
terms_final = df_base.alt2_terms.to_list()

In [ ]:
terms_final_flat = []
for i in range(len(terms_final)):
    for token in terms_final[i]:
        terms_final_flat.append(token)

In [ ]:
len(set(terms_final_flat))

In [ ]:
terms_final_flat = Counter(terms_final_flat)
sort_orders_terms_final = sorted(terms_final_flat.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_terms_final:
    print(i[0], i[1])

In [ ]:
df_base.head()

In [ ]:
## v1.2: Store terms processed by stanza (Spanish)
with open('./output/terms_lemmat_tcs_and_loans_2020-11-12_spanish.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(df_base.alt2_terms, filehandle)

# **************************************************************************************************************** #
<br>
<br>
<br>

## Spanish: NLP Token extraction and processing

In [ ]:
import gensim

In [ ]:
# Tokenize Sentences and Clean
def sent_to_words(sentences):
    for sent in sentences:
        #sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        #sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=False) #modificado
        yield(sent) 

In [ ]:
# Convert to list
data = df_base['extracted'].values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
pprint(data_words[:1])

In [ ]:
# Main functions
#def remove_stopwords(texts, stop_words):
#    return [[word for word in gensim.utils.simple_preprocess(str(doc), deacc=False) if word not in stop_words] for doc in texts]
#
##
#def lemmatization(texts, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP']):
#    texts_out = []
#    for sent in texts:
#        doc = nlp_es(" ".join(sent)) 
#        texts_out.append([token.lemma_ for token in doc if (len(token) > 1 and token.pos_ in allowed_postags)])
#    # remove stopwords once more after lemmatization
#    texts_out = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]   
#    return texts_out

In [ ]:
#stop_words

In [ ]:
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words, stop_words)
#
## Data Lemmatized
#data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP'])

In [ ]:
#
def lemmatization(texts, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP']):
    texts_out = []
    for sent in texts:
        doc = stNLP(" ".join(sent)) 
        texts_out.append([word.lemma.lower() for sent in doc.sentences for word in sent.words if word.pos in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]   
    return texts_out

In [ ]:
%%time
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words, stop_words)

# Data Lemmatized
data_lemmatized = lemmatization(data_words, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP'])

In [ ]:
# Word evaluation:
word_stats_only_tokens = []
for i in range(len(data_lemmatized)):
    for token in data_lemmatized[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats_only_tokens.append(token)

In [ ]:
len(set(word_stats_only_tokens))

### Merge tokens and terms/n-Grams

In [ ]:
df_base.head()

In [ ]:
data_lemmatized[550]

In [ ]:
df_base.reset_index(drop=True, inplace=True)

In [ ]:
# obtained terms/n-grams are added to the dataset
data_lemmatized_full = []
for index, row in df_base.iterrows():
    data_lemmatized_full.append(data_lemmatized[index] + df_base.alt2_terms[index])

#  ~~ ****** ~~ 

In [ ]:
# Word evaluation:
word_stats = []
for i in range(len(data_lemmatized_full)):
    for token in data_lemmatized_full[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats.append(token)

In [ ]:
word_stats = Counter(word_stats)

sort_orders = sorted(word_stats.items(), key=lambda x: x[1], reverse=True)

for i in sort_orders:
    print(i[0], i[1])

In [ ]:
df_base['data_lemmatized_full'] = data_lemmatized_full

# **************************************************************************************************************** #
<br>
<br>
<br>

## Store results

In [ ]:
import pickle

In [ ]:
df_base.head()

In [ ]:
## v1.2: Store data_lemmatized_full ONLY TOKENS (Spanish)
with open('./output/data_lemmatized_TOKENS_es_2020-11-12.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized, filehandle)

In [ ]:
## v1.2: Store data_lemmatized_full (Spanish)
with open('./output/data_lemmatized_full_es_2020-11-12.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized_full, filehandle)

In [ ]:
# v1.2: Store df_base, in Spanish, containing terms
f_df_base_es = 'nlp_df_base_2020-11-12_spanish.joblib'
joblib.dump(df_base[['doc_type', 'language', 'FK_OPERATION_ID', 'OPERATION_NUMBER',
       'DOCUMENT_ID', 'DOCUMENT_REFERENCE', 'Document_Name',
       'extracted', 'alt2_list_terms_base', 'alt2_list_terms', 'alt2_terms', 'data_lemmatized_full']], './output/' + f_df_base_es + '.bz2', compress=('bz2', 3))#

In [ ]:
%%time
# v1.2: Store df_base, in Spanish, containing terms
f_df_base_es2 = 'nlp_2020-11-12_spacy_annotated_spanish.joblib'
joblib.dump(df_base[['OPERATION_NUMBER', 'DOCUMENT_REFERENCE', 'Document_Name', 'textacy_processing']], './output/' + f_df_base_es2 + '.bz2', compress=('bz2', 3))#

# **************************************************************************************************************** #
<br>
<br>
<br>

# **************************************************************************************************************** #
<br>
<br>
<br>

In [ ]:
## v1.1: Store data_lemmatized_full (Spanish)
with open('./output/data_lemmatized_full_es_2020-11-10.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized_full, filehandle)

In [ ]:
# v1.1: Store df_base, in Spanish, containing terms
f_df_base_es = 'nlp_df_base_2020-11-10_spanish.joblib'
joblib.dump(df_base[['doc_type', 'language', 'FK_OPERATION_ID', 'OPERATION_NUMBER',
       'DOCUMENT_ID', 'DOCUMENT_REFERENCE', 'DESCRIPTION', 'Document_Name',
       'extracted', 'terms', 'data_lemmatized_full']], './output/' + f_df_base_es + '.bz2', compress=('bz2', 3))#

In [ ]:
len(df_base.OPERATION_NUMBER)

In [ ]:
#####

In [ ]:
## v1.0: Store data_lemmatized_full (Spanish)
with open('./output/data_lemmatized_full_es_2020-10-20.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized_full, filehandle)

In [ ]:
# v1.0: Store df_base, in Spanish, containing terms
f_df_base_es = 'nlp_df_base_2020-10-20_spanish.joblib'
joblib.dump(df_base[['doc_type', 'language', 'FK_OPERATION_ID', 'OPERATION_NUMBER',
       'DOCUMENT_ID', 'DOCUMENT_REFERENCE', 'DESCRIPTION', 'Document_Name',
       'extracted', 'terms']], './output/' + f_df_base_es + '.bz2', compress=('bz2', 3))#

# **************************************************************************************************************** #
<br>
<br>
<br>

# English Language Documents

In [ ]:
# Load operations' filters: ENGLISH documents
df_filters_1_en = pd.read_excel('./input/Lista de Operaciones con Documento Encontrado-ES-EN.xlsx', sheet_name='EN')
df_filters_1_en['OPERATION_NUMBER'] = df_filters_1_en['OPERATION_NUMBER'].str.strip()

In [ ]:
df_filters_en = pd.concat([df_filters_1_en[['OPERATION_NUMBER']], df_filters_2[['OPERATION_NUMBER']], \
                        df_filters_3[['OPERATION_NUMBER']]], ignore_index=True)

In [ ]:
df_filters_en

In [ ]:
# filter by selected operations:
df_base_en = data_base[data_base['OPERATION_NUMBER'].isin(df_filters_en['OPERATION_NUMBER'])]
# select the Spanish documents:
df_base_en = df_base_en[df_base_en['language'] == 'en']

##### Check for duplicates:

In [ ]:
len(df_base_en.OPERATION_NUMBER.unique())

In [ ]:
df_base_en[df_base_en.duplicated(subset=['OPERATION_NUMBER'])]

In [ ]:
df_base_en[df_base_en.OPERATION_NUMBER == 'BR-T1415']

In [ ]:
df_base_en.drop([742], inplace=True)

In [ ]:
df_base_en[df_base_en.OPERATION_NUMBER == 'UR-L1140']

In [ ]:
df_base_en.drop([1392], inplace=True)

In [ ]:
df_base_en[df_base_en.OPERATION_NUMBER == 'UR-L1156']

In [ ]:
df_base_en.drop([1827], inplace=True)

In [ ]:
df_base_en.reset_index(drop=True, inplace=True)

In [ ]:
### ~~ ### ~~ ### ~~ ###

In [ ]:
df_base_en.shape

In [ ]:
#!python -m spacy download en_core_web_lg

## English: NLP n-Gram Analysis - using Textacy bag-of-terms

In [ ]:
import spacy, en_core_web_lg
nlp_en = spacy.load('en_core_web_lg', disable=['ner'])

#### Stop Words Setup

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# Spacy stop_words
stop_words.extend(nlp_en.Defaults.stop_words)

In [ ]:
# custom stop_words:
stop_words.extend(['ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'ill', 'descripción', 'componente', 'objetivo', 'ct', 'mailto', 'país', 'millón', 'millones', \
                   'año', 'años', 'dólar', 'dolar', 'dólares', 'si', 'bid', 'us', 'oc', 'gn', 'tc', 'atn', 'opc', 'pib', 'ar', 'br', 'uy', 'cl', 'co', \
                   'cclip', 'pbl', 'uis', 'ab', 'org', 'pr', 'bo', 'bl', 'pe', 'ec', 'ja', 'mx', 'ca', 'gu', 'su', 'ho', 'hn', 'mr', 'rg', 'ee', 'uu', \
                   'cr', 'tdr', 'rn', 'nº', 'usd', 'gy', 've', 'et', 'the', 'for', 'to', 'grt', 'fm', 'pr', 'pa', 'ni', 'aa', 'es', 'sp', \
                   'inglés', 'cty', 'nv', 'profisco', 'asimismo', 'actual', 'costo', 'resultar', 'esperar', 'ejecutar', 'unidad', 'agencia', 'justificación', \
                   'véase', 'ct', 'loan', 'paragraph', 'lac', 'optional'])

stop_words = list(set(stop_words))

### 1. Textacy processing

In [ ]:
# Load Spacy Spanish model:
en = textacy.load_spacy_lang('en_core_web_lg')

In [ ]:
# Textacy processing on extracted text: 
df_base_en['textacy_processing'] = ''

In [ ]:
%%time
for index, row in df_base_en.iterrows():
    #print('Processing index:', str(index))
    df_base_en.at[index, 'textacy_processing'] = textacy.make_spacy_doc(df_base_en.extracted[index].lower(), lang=en)
    
df_base_en.head()

In [ ]:
type(df_base_en.textacy_processing[0])

### 2. List of Terms (Bag-of-Terms): n-Grams extraction

#### 2.2. List of Terms Generation

In [ ]:
df_base_en['list_of_terms'] = ''

In [ ]:
%%time
for index, row in df_base_en.iterrows():
    #print('Processing index:', str(index))
    
    #generate terms:
    terms_list_en = df_base_en['textacy_processing'][index]._.to_terms_list(ngrams=(2, 3, 4, 5, 6), entities=False, normalize="lemma", weighting="count", as_strings=True, filter_stops=True, filter_punct=True, filter_nums=True, include_pos=['PROPN', 'NOUN', 'ADJ', 'ADP'], min_freq=2)
    
    #replace blanks with '_':
    resultado_pre_en = Counter([(item.lower()).replace(' ', '_') for item in terms_list_en])
    
    #select terms that appear 2 or more times, convert to list including count and store:
    df_base_en.at[index, 'list_of_terms'] = [k for (k,v) in resultado_pre_en.items() for count in range(v) if v > 1 ]


#### 2.3. List of Terms: Clean-up

##### Clean-up

In [ ]:
df_base_en

In [ ]:
terms_result = df_base_en.list_of_terms.to_list()

In [ ]:
terms_grams = []
for i in range(len(terms_result)):
    for token in terms_result[i]:
        terms_grams.append(token)

In [ ]:
(len((terms_grams)),len(set(terms_grams)))

In [ ]:
terms_grams = Counter(terms_grams)
sort_orders_terms = sorted(terms_grams.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_terms:
    print(i[0], i[1])

In [ ]:
len(sort_orders_terms)

In [ ]:
terms_to_remove_en = []
for i in range(0,len(sort_orders_terms)):
    if (sort_orders_terms[i][0].endswith('_iv') or sort_orders_terms[i][0].endswith('_ii') or sort_orders_terms[i][0].endswith('_us$') or sort_orders_terms[i][0].endswith('/') \
        or sort_orders_terms[i][0].endswith('.') or sort_orders_terms[i][0].endswith('_i') or sort_orders_terms[i][0].endswith('_iii') or sort_orders_terms[i][0].endswith('_”')\
        or sort_orders_terms[i][0].endswith('_a') or sort_orders_terms[i][0].startswith('“_') or sort_orders_terms[i][0].startswith('f._') or sort_orders_terms[i][0].startswith('a._') \
       or sort_orders_terms[i][0].startswith('b._') or sort_orders_terms[i][0].startswith('c._') or sort_orders_terms[i][0].startswith('d._') \
       or sort_orders_terms[i][0].startswith('e._') or sort_orders_terms[i][0].startswith('v._') or sort_orders_terms[i][0].startswith('i._') \
       or sort_orders_terms[i][0].startswith('g._') or sort_orders_terms[i][0].startswith('iv._') or sort_orders_terms[i][0].startswith('&_') \
       or sort_orders_terms[i][0].startswith('actividad/_') or sort_orders_terms[i][0].startswith('ct_') or sort_orders_terms[i][0].startswith('atn_/') or sort_orders_terms[i][0].startswith('us$_')\
        or sort_orders_terms[i][0].startswith('/_') or sort_orders_terms[i][0].startswith('ii.') or sort_orders_terms[i][0].startswith('iii_') or sort_orders_terms[i][0].startswith('iv_')\
       or sort_orders_terms[i][0].startswith('a_')):
       
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
len(set(terms_to_remove))

In [ ]:
# remove grmas containing digits:
for i in range(0,len(sort_orders_terms)):
    if (num_there(sort_orders_terms[i][0]) and not 'covid' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if ('mention' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if ('following' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if ('paragraph' in sort_orders_terms[i][0]):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
for i in range(0,len(sort_orders_terms)):
    if sort_orders_terms[i][0].endswith('_cabo') or sort_orders_terms[i][0].startswith('cabo_'):
        print(sort_orders_terms[i][0], sort_orders_terms[i][1])
        #terms_to_remove_en.append(sort_orders_terms[i][0])

In [ ]:
for i in sort_orders_terms:
    if 'take' in i[0]:
        print(i[0], i[1])
        #terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'give'  in i[0]:
        print(i[0], i[1])
        #terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'term'  in i[0]:
        print(i[0], i[1])
        #terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if '▪' in i[0]:
        print(i[0], i[1])
        terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'component' in i[0]:
        print(i[0], i[1])
        terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'executing'  in i[0]:
        print(i[0], i[1])
        terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'efficient'  in i[0]:
        print(i[0], i[1])
        #terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'consult'  in i[0]:
        print(i[0], i[1])
        terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'wait'  in i[0]:
        print(i[0], i[1])
        #terms_to_remove_en.append(i[0])

In [ ]:
for i in sort_orders_terms:
    if 'actual'  in i[0]:
        print(i[0], i[1])
        terms_to_remove_en.append(i[0])

In [ ]:
terms_to_remove_en = terms_to_remove_en + ['componente_iia', 'asistencia_técnico', 'capacidad_institucional', 'estrategia_institucional', 'área_transversal', 'estructurar_ejecución', \
 'componente_descripción', 'operación_préstamo', 'organismo_ejecutor', 'américa_latina', 'cooperación_técnico', 'cooperación_técnica', 'operación_de_cooperación', 'agencia_ejecutora', \
  'operación_de_cooperación_técnico', 'agencia_ejecutor', 'nivel_mundial', 'organismo_ejecutor', 'unidad_ejecutor', 'resultar_esperar', 'producto_esperar', 'esperar_del_componente', \
                                    'resultados_esperar', 'resultar_esperar_del_componente', 'principal_resultar_esperar', 'poner_en_funcionamiento', 'problema_específico', \
                                    'nivel_nacional', 'autoridad_nacional', 'presentar_operación', 'resultar_anual', 'estructurar_organizacional', 'gobernar_central', 'tomar_de_decisión',\
                                    'adicional_con_programar', 'documento_de_marco_sectorial', 'adquisición_de_insumo', 'proyectar_pilotar', 'alto_impactar', 'efectividad_comparar', \
                                    'modelar_de_negociar', 'término_de_referenciar', 'aumentar_sostener', 'aumentar_inicial', 'aumentar_del_nivel', 'lección_aprender', 'et_al', 'new_area', \
                                    'technical_assistance', 'optional_link', 'good_practice', 'sector_framework', 'action_plan', 'short_term', 'long_term', 'medium_term', 'year_implementation'\
                                    'tc_resource', 'year_action', 'grant_operation', 'year_of_age'    ]

In [ ]:
terms_to_remove_en = list(set(terms_to_remove_en))

In [ ]:
len(terms_to_remove_en)

In [ ]:
index=121
#df_base['list_of_terms'][1729]
[word for word in df_base_en['list_of_terms'][index] if word not in terms_to_remove_en and '_' in word]

#### 2.5. Remove selected terms

In [ ]:
df_base_en['terms'] = ''

In [ ]:
%%time
for index, row in df_base_en.iterrows():
    #print('Processing index:', str(index))
    df_base_en.at[index, 'terms'] = [word for word in df_base_en['list_of_terms'][index] if word not in terms_to_remove_en and '_' in word]

#### 2.6. Check results

In [ ]:
terms_final = df_base_en.terms.to_list()

In [ ]:
terms_final_flat = []
for i in range(len(terms_final)):
    for token in terms_final[i]:
        terms_final_flat.append(token)

In [ ]:
len(set(terms_final_flat))

In [ ]:
terms_final_flat = Counter(terms_final_flat)
sort_orders_terms_final = sorted(terms_final_flat.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders_terms_final:
    print(i[0], i[1])

# **************************************************************************************************************** #
<br>
<br>
<br>

## NLP Token extraction and processing

In [ ]:
df_base_en.head()

In [ ]:
# Convert to list
data_en = df_base_en['extracted'].values.tolist()
data_words_en = list(sent_to_words(data_en))

In [ ]:
pprint(data_words_en[:1])

In [ ]:
# Main functions
def remove_stopwords_en(texts, stop_words):
    return [[word for word in gensim.utils.simple_preprocess(str(doc), deacc=False) if word not in stop_words] for doc in texts]

#
def lemmatization_en(texts, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP']):
    texts_out = []
    for sent in texts:
        doc = nlp_en(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if (len(token) > 1 and token.pos_ in allowed_postags)])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]   
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops_en = remove_stopwords(data_words_en, stop_words)

# Data Lemmatized
data_lemmatized_en = lemmatization_en(data_words_nostops_en, allowed_postags=['PROPN', 'NOUN', 'ADJ', 'ADP'])

In [ ]:
# Word evaluation:
word_stats_only_tokens_en = []
for i in range(len(data_lemmatized_en)):
    for token in data_lemmatized_en[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats_only_tokens_en.append(token)

In [ ]:
len(set(word_stats_only_tokens_en))

### Merge tokens and terms/n-Grams

In [ ]:
df_base_en.reset_index(drop=True, inplace=True)

In [ ]:
# obtained terms/n-grams are added to the dataset
data_lemmatized_full_en = []
for index, row in df_base_en.iterrows():
    data_lemmatized_full_en.append(data_lemmatized_en[index] + df_base_en.terms[index])

#  ~~ ****** ~~ 

In [ ]:
# Word evaluation:
word_stats = []
for i in range(len(data_lemmatized_full_en)):
    for token in data_lemmatized_full_en[i]:
        #if '_' in token:
        #    print(str(i),token)
        word_stats.append(token)

In [ ]:
word_stats = Counter(word_stats)

sort_orders = sorted(word_stats.items(), key=lambda x: x[1], reverse=True)

for i in sort_orders:
    print(i[0], i[1])

# **************************************************************************************************************** #
<br>
<br>
<br>

## Store results

In [ ]:
## v1.0: Store data_lemmatized_full (English)
with open('./output/data_lemmatized_full_en_2020-10-20.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_lemmatized_full_en, filehandle)

In [ ]:
# v1.0: Store df_base, in English, containing terms
f_df_base_en = 'nlp_df_base_2020-10-20_english.joblib'
joblib.dump(df_base_en[['doc_type', 'language', 'FK_OPERATION_ID', 'OPERATION_NUMBER',
       'DOCUMENT_ID', 'DOCUMENT_REFERENCE', 'DESCRIPTION', 'Document_Name',
       'extracted', 'terms']], './output/' + f_df_base_en + '.bz2', compress=('bz2', 3))#

# **************************************************************************************************************** #
<br>
<br>
<br>

In [ ]:
#'''
# **************************************************************************************************************** #
# ********************************************  Version Control  ************************************************* #
# **************************************************************************************************************** #
  
#   Version:            Date:                User:                   Change:                                       

#   - 1.2           01/14/2020        Emiliano Colina    - latest documents
#

#   - 1.1           11/10/2020        Emiliano Colina    - Updated version, removing stop-words from terms,      
#                                                         tested inflector, Stanza chosed 


#   - 1.0           10/19/2020        Emiliano Colina    - Initial version, to include a separate layer for NLP     
#                                                         processing on a dedicated notebook

#
# **************************************************************************************************************** #
#'''
